In [39]:
import pandas as pd
import numpy as np
import fiona
import geopandas as gpd
import os

Two runs: run310 NP ("carb_test_np"), run313 P ("carb_test_p")

# 1. parcel_subzone and logsum files from TM

In [2]:
# parcel_subzone lookup
parcel_subzone_lookup = pd.read_csv(r'X:\bayarea_urbansim\data\2020_08_17_parcel_to_taz1454sub.csv',
                                    usecols=['taz_sub', 'ZONE_ID', 'PARCEL_ID', 'county'])
# print(parcel_subzone_lookup_raw.dtypes)

# parcel_subzone_lookup = parcel_subzone_lookup_raw.copy()
# parcel_subzone_lookup['PARCEL_ID'] = parcel_subzone_lookup['PARCEL_ID'].apply(lambda x: int(round(x)))

# some parcels have no taz_sub
parcel_lookup_nosubzone = parcel_subzone_lookup.loc[parcel_subzone_lookup.taz_sub.isnull()]
print('parcel_subzone_lookup has {} parcels, of which {} are missing subzone'.format(
    parcel_subzone_lookup.shape[0],
    parcel_lookup_nosubzone.shape[0]))

parcel_subzone_lookup has 1956208 parcels, of which 1514 are missing subzone


In [3]:
# parcel_id to SD name lookup
p_taz = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\p10_parcel_TAZ_SD.csv')
p_taz.rename(columns = {'parcel_id': 'PARCEL_ID'}, inplace=True)

print(p_taz.shape)
display(p_taz.head())

# add SD_NAME to parcel_subzone lookup
parcel_subzone_lookup = parcel_subzone_lookup.merge(p_taz, on='PARCEL_ID', how='outer')
display(parcel_subzone_lookup)

(1956208, 4)


,PARCEL_ID,TAZ,SUPERD,SD_NAME
0,229116,725.0,15,Tri Valley
1,244166,715.0,15,Tri Valley
2,202378,820.0,17,S Leandro Hayward
3,2004420,1401.0,31,Northern Sonoma
4,340332,763.0,16,Fremont


,PARCEL_ID,ZONE_ID,county,taz_sub,TAZ,SUPERD,SD_NAME
0,229116,725,ala,725b,725.0,15,Tri Valley
1,244166,715,ala,715a,715.0,15,Tri Valley
2,202378,820,ala,820a,820.0,17,S Leandro Hayward
3,2004420,1401,son,1401c,1401.0,31,Northern Sonoma
4,340332,763,ala,763a,763.0,16,Fremont
...,...,...,...,...,...,...,...
1956203,580747,1193,cnc,1193a,1193.0,24,East Contra Costa
1956204,594355,1111,cnc,1111a,1111.0,21,Martinez Concord
1956205,804156,1411,mar,1411b,1411.0,32,Northern Marin
1956206,646401,1176,cnc,1176a,1176.0,23,S Ramon Danville


In [4]:
# logsum inputs - 2050
logsum_input_dir = 'M:\\Application\\Model One\\RTP2021\\SensitivityTests_CARB'

NP_logsum_2050_dir = os.path.join(logsum_input_dir, r'2050_TM152_SEN_NoProject_01b\OUTPUT\logsums\ForUrbanSim')
accessMarket_NP_2050_raw = pd.read_csv(os.path.join(NP_logsum_2050_dir, 'AccessibilityMarkets_2050_s28.csv'))
mandatory_NP_2050_raw = pd.read_csv(os.path.join(NP_logsum_2050_dir, 'mandatoryAccessibilities_2050_s28.csv'))
nonMandatory_NP_2050_raw = pd.read_csv(os.path.join(NP_logsum_2050_dir, 'nonMandatoryAccessibilities_2050_s28.csv'))

P_logsum_2050_dir = os.path.join(logsum_input_dir, r'2050_TM152_SEN_Project_01\OUTPUT\logsums\ForUrbanSim')
accessMarket_P_2050_raw = pd.read_csv(os.path.join(P_logsum_2050_dir, 'AccessibilityMarkets_2050_s28.csv'))
mandatory_P_2050_raw = pd.read_csv(os.path.join(P_logsum_2050_dir, 'mandatoryAccessibilities_2050_s28.csv'))
nonMandatory_P_2050_raw = pd.read_csv(os.path.join(P_logsum_2050_dir, 'nonMandatoryAccessibilities_2050_s28.csv'))

display(accessMarket_NP_2050_raw.head())
display(mandatory_NP_2050_raw.head())


# logsum inputs - 2035
logsum_input_dir = 'M:\\Application\\Model One\\RTP2021\\SensitivityTests_CARB'

NP_logsum_2035_dir = os.path.join(logsum_input_dir, r'2035_TM152_SEN_NoProject_01b\OUTPUT\logsums\ForUrbanSim')
accessMarket_NP_2035_raw = pd.read_csv(os.path.join(NP_logsum_2035_dir, 'AccessibilityMarkets_2035_s28.csv'))
mandatory_NP_2035_raw = pd.read_csv(os.path.join(NP_logsum_2035_dir, 'mandatoryAccessibilities_2035_s28.csv'))
nonMandatory_NP_2035_raw = pd.read_csv(os.path.join(NP_logsum_2035_dir, 'nonMandatoryAccessibilities_2035_s28.csv'))

P_logsum_2035_dir = os.path.join(logsum_input_dir, r'2035_TM152_SEN_Project_01\OUTPUT\logsums\ForUrbanSim')
accessMarket_P_2035_raw = pd.read_csv(os.path.join(P_logsum_2035_dir, 'AccessibilityMarkets_2035_.csv'))
mandatory_P_2035_raw = pd.read_csv(os.path.join(P_logsum_2035_dir, 'mandatoryAccessibilities_2035_.csv'))
nonMandatory_P_2035_raw = pd.read_csv(os.path.join(P_logsum_2035_dir, 'nonMandatoryAccessibilities_2035_.csv'))

# display(accessMarket_NP_2035_raw.head())
# display(mandatory_NP_2035_raw.head())

,taz,walk_subzone,walk_subzone_label,incQ,incQ_label,autoSuff,autoSuff_label,hasAV,num_persons,num_workers,num_workers_students
0,1,1,Short-walk to transit,1,lowInc,0,0_autos,0,20,2,2
1,1,1,Short-walk to transit,1,lowInc,2,autos_ge_workers,0,14,0,0
2,1,1,Short-walk to transit,2,medInc,0,0_autos,0,4,0,0
3,1,1,Short-walk to transit,2,medInc,2,autos_ge_workers,0,8,0,0
4,1,1,Short-walk to transit,3,highInc,0,0_autos,0,10,8,10


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_noAV,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone
0,11.802405,10.896873,10.885533,10.303212,10.275660,12.185586,11.220437,11.216252,10.624694,10.612716,...,10.598943,10.582088,12.500878,11.490281,11.485653,10.947967,10.937449,0,1,0
1,12.595845,11.100052,11.094865,10.359887,10.337461,13.055068,11.486221,11.484006,10.707514,10.707516,...,10.685218,10.685245,13.363009,11.776068,11.773426,11.032995,11.033057,1,1,1
2,12.418061,11.018443,11.011700,10.331777,10.331725,12.867915,11.382873,11.380087,10.667001,10.666922,...,10.643279,10.627807,13.173682,11.665842,11.662128,10.991492,10.981602,2,1,2
3,11.789812,10.883476,10.872151,10.286931,10.258883,12.170130,11.206445,11.201583,10.607694,10.595546,...,10.590962,10.574059,12.486032,11.475791,11.470766,10.931263,10.920573,3,2,0
4,12.574998,11.078485,11.073135,10.340624,10.317442,13.017905,11.458837,11.456473,10.684883,10.684877,...,10.670213,10.670212,13.319900,11.745857,11.743056,11.009636,11.009670,4,2,1


# 2. break down BAUS cml, cnml, and comb_logsum functions

In [5]:
# modify logsum files from Travel Model

# recode 'taz' and 'subzone' into 'taz_sub'
def create_subzone(logsum_input_df_raw):
    logsum_input_df = logsum_input_df_raw.copy()
    logsum_input_df.loc[logsum_input_df.subzone == 0, 'subzone'] = 'c'  # no walk
    logsum_input_df.loc[logsum_input_df.subzone == 1, 'subzone'] = 'a'  # short walk
    logsum_input_df.loc[logsum_input_df.subzone == 2, 'subzone'] = 'b'  # long walk
    logsum_input_df['taz_sub'] = logsum_input_df.taz.astype('str') + logsum_input_df.subzone
    return logsum_input_df

# modify accessibilityMarket input cateogries to be consistent with logsum
def modify_accessMarket(accessMarket_df_raw):
    accessMarket_df = accessMarket_df_raw.copy()
    accessMarket_df['AV'] = accessMarket_df['hasAV'].apply(lambda x: 'AV' if x == 1 else 'noAV')
    accessMarket_df['label'] = (accessMarket_df['incQ_label'] + '_' + accessMarket_df['autoSuff_label'] + '_' + accessMarket_df['AV'])
    accessMarket_df = accessMarket_df.groupby('label').sum()
    accessMarket_df['prop'] = accessMarket_df['num_persons'] / accessMarket_df['num_persons'].sum()
    accessMarket_df = accessMarket_df[['prop']].transpose().reset_index(drop=True)
    return accessMarket_df

In [6]:
# calculated mandatory_logsum, non-mandatory_logsum
# note: in BAUS, 'tab_sub' from "2020_08_17_parcel_to_taz1454sub.csv" is merged into 'parcels' as a new column 'subzone' in variables.py

def cml_step1(mandatory_accessibility, accessibilities_segmentation):
    """
    step 1: scale mandatory accessibility - calculate relative logsum, then scale by 1/0.0134
    """
    mandatory_accessibility_scale = mandatory_accessibility.copy()
#     display(mandatory_accessibility_scale.head())
    for col in mandatory_accessibility_scale.columns[~mandatory_accessibility_scale.columns.isin(['destChoiceAlt',
                                                        'taz', 'subzone',
                                                        'weighted_sum'])]:
        if col in accessibilities_segmentation.columns:
            mandatory_accessibility_scale[col] = \
                ((mandatory_accessibility_scale[col] - mandatory_accessibility_scale[col].min()) / 0.0134)
#     display(mandatory_accessibility_scale.head())
    return mandatory_accessibility_scale

def cml_step2(mandatory_accessibility_scale, accessibilities_segmentation):
    """
    step 2: calculate weighted sum - weighting by region-wide market segments
    """
    mandatory_accessibility_sum = mandatory_accessibility_scale.copy()
    cols_to_sum = []
    for col in mandatory_accessibility_sum.columns[~mandatory_accessibility_sum.columns.isin(['destChoiceAlt',
                                                                                      'taz', 'subzone',
                                                                                      'weighted_sum'])]:
        if col in accessibilities_segmentation.columns:
            mandatory_accessibility_sum[col] = mandatory_accessibility_sum[col] * accessibilities_segmentation.loc[0, col]
            cols_to_sum.append(col)
    mandatory_accessibility_sum['weighted_sum'] = mandatory_accessibility_sum[cols_to_sum].sum(axis=1)
    return mandatory_accessibility_sum

def cml_step3(parcels, mandatory_accessibility_sum):
    """
    step 3: join scaled and weighted mandatory logsum to parcel,
            fill na with -1: na representing parcels without sub_taz or failed to join to a sub_taz
    """
    parcels_cml = parcels[['PARCEL_ID', 'taz_sub', 'TAZ', 'SD_NAME']].merge(
                  mandatory_accessibility_sum[['taz_sub', 'weighted_sum']], on='taz_sub', how='left')
    parcels_cml.rename(columns={'weighted_sum': 'cml_raw'}, inplace=True)
    
    print('before filling na with -1, {} parcels have no logsum_cml'.format(
        parcels_cml.loc[parcels_cml.cml_raw.isnull()].shape[0]))
    
    parcels_cml['cml'] = parcels_cml['cml_raw']
    parcels_cml['cml'].fillna(-1, inplace=True)
    return parcels_cml

    
    
def cnml_step1(nonMandatory_accessibility, accessibilities_segmentation):
    """
    step 1: scale nonMandatory accessibility - calculate relative logsum, then scale by 1/0.0134
    """
    nonMandatory_accessibility_scale = nonMandatory_accessibility.copy()
#     display(nonMandatory_accessibility_scale.head())
    for col in nonMandatory_accessibility_scale.columns[~nonMandatory_accessibility_scale.columns.isin(['destChoiceAlt',
                                                        'taz', 'subzone',
                                                        'weighted_sum'])]:
        if col in accessibilities_segmentation.columns:
            nonMandatory_accessibility_scale[col] = \
                ((nonMandatory_accessibility_scale[col] - nonMandatory_accessibility_scale[col].min()) / 0.0175)
#     display(nonMandatory_accessibility_scale.head())
    return nonMandatory_accessibility_scale

def cnml_step2(nonMandatory_accessibility_scale, accessibilities_segmentation):
    """
    step 2: calculate weighted sum - weighting by region-wide market segments
    """
    nonMandatory_accessibility_sum = nonMandatory_accessibility_scale.copy()
    cols_to_sum = []
    for col in nonMandatory_accessibility_sum.columns[~nonMandatory_accessibility_sum.columns.isin(['destChoiceAlt',
                                                                                      'taz', 'subzone',
                                                                                      'weighted_sum'])]:
        if col in accessibilities_segmentation.columns:
            nonMandatory_accessibility_sum[col] = nonMandatory_accessibility_sum[col] * accessibilities_segmentation.loc[0, col]
            cols_to_sum.append(col)
    nonMandatory_accessibility_sum['weighted_sum'] = nonMandatory_accessibility_sum[cols_to_sum].sum(axis=1)
    return nonMandatory_accessibility_sum

def cnml_step3(parcels, nonMandatory_accessibility_sum):
    """
    step 3: join scaled and weighted mandatory logsum to parcel,
            fill na with -1: na representing parcels without sub_taz or failed to join to a sub_taz
    """
    parcels_cnml = parcels[['PARCEL_ID', 'taz_sub', 'TAZ', 'SD_NAME']].merge(
                   nonMandatory_accessibility_sum[['taz_sub', 'weighted_sum']], on='taz_sub', how='left')   
    parcels_cnml.rename(columns={'weighted_sum': 'cnml_raw'}, inplace=True)
    
    print('before filling na with -1, {} parcels have no logsum_cnml'.format(
        parcels_cnml.loc[parcels_cnml.cnml_raw.isnull()].shape[0]))
    
    parcels_cnml['cnml'] = parcels_cnml['cnml_raw']
    parcels_cnml['cnml'].fillna(-1, inplace=True)
    
    return parcels_cnml


In [7]:
# calculate combo logsum
def combo_logsum(cml, cnml):
    # first, sum up cml and cnml
    parcel_logsum_df = cml.merge(cnml, on=['PARCEL_ID', 'taz_sub', 'TAZ', 'SD_NAME'], how='outer')
    parcel_logsum_df['combo_logsum_raw'] = parcel_logsum_df['cml'] + parcel_logsum_df['cnml']
    
    # second, scale by minusing 170
#     parcel_logsum_df['combo_logsum_minus170'] = parcel_logsum_df['combo_logsum_raw']
    parcel_logsum_df['combo_logsum_minus170'] = parcel_logsum_df['combo_logsum_raw'] - 170
    
    # third, fill na with 1
    parcel_logsum_df['combo_logsum_minus170_fillNA'] = parcel_logsum_df['combo_logsum_minus170']
    parcel_logsum_df.loc[parcel_logsum_df.combo_logsum_minus170_fillNA <= 0, 'combo_logsum_minus170_fillNA'] = 1
    
    return parcel_logsum_df


# 3. replicate combo_logsum calculation and compare with BAUS output

## 3-1.  Travel Model 2050 NP logsum

In [8]:
# first, modify input files
accessMarket_NP_2050 = modify_accessMarket(accessMarket_NP_2050_raw)
mandatory_NP_2050 = create_subzone(mandatory_NP_2050_raw)
nonMandatory_NP_2050 = create_subzone(nonMandatory_NP_2050_raw)

In [9]:
# second, calculate mandatory logsums by parcel
display(mandatory_NP_2050)

mandatory_NP_2050_scale = cml_step1(mandatory_NP_2050, accessMarket_NP_2050)
display(mandatory_NP_2050_scale)

mandatory_NP_2050_sum = cml_step2(mandatory_NP_2050_scale, accessMarket_NP_2050)
display(mandatory_NP_2050_sum)

parcels_cml_NP_2050 = cml_step3(parcel_subzone_lookup, mandatory_NP_2050_sum)
parcels_cml_NP_2050

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,11.802405,10.896873,10.885533,10.303212,10.275660,12.185586,11.220437,11.216252,10.624694,10.612716,...,10.582088,12.500878,11.490281,11.485653,10.947967,10.937449,0,1,c,1c
1,12.595845,11.100052,11.094865,10.359887,10.337461,13.055068,11.486221,11.484006,10.707514,10.707516,...,10.685245,13.363009,11.776068,11.773426,11.032995,11.033057,1,1,a,1a
2,12.418061,11.018443,11.011700,10.331777,10.331725,12.867915,11.382873,11.380087,10.667001,10.666922,...,10.627807,13.173682,11.665842,11.662128,10.991492,10.981602,2,1,b,1b
3,11.789812,10.883476,10.872151,10.286931,10.258883,12.170130,11.206445,11.201583,10.607694,10.595546,...,10.574059,12.486032,11.475791,11.470766,10.931263,10.920573,3,2,c,2c
4,12.574998,11.078485,11.073135,10.340624,10.317442,13.017905,11.458837,11.456473,10.684883,10.684877,...,10.670212,13.319900,11.745857,11.743056,11.009636,11.009670,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,7.886158,7.911058,7.749130,7.752692,7.752690,7.908494,7.879234,7.812608,7.668451,7.668445,...,8.352696,9.728498,8.969375,8.945592,8.749888,8.749915,4357,1453,a,1453a
4358,7.888928,7.911040,7.749070,7.752678,7.752689,7.907697,7.879506,7.812867,7.668435,7.567725,...,8.352690,9.721445,8.969301,8.945653,8.749918,8.714520,4358,1453,b,1453b
4359,6.833059,6.702897,6.492826,6.583241,6.583235,6.038036,6.126233,6.010848,5.989114,5.989112,...,6.850659,8.169195,7.516215,7.476047,7.369251,7.369235,4359,1454,c,1454c
4360,6.830547,6.702862,6.492789,6.583233,6.583234,6.025141,6.126268,6.010541,5.989114,5.839668,...,6.749752,8.165688,7.516075,7.476065,7.369264,7.318302,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,840.184254,787.158881,816.592090,752.081567,783.083060,874.878731,822.539925,835.212836,785.261866,797.790448,...,642.476418,573.055224,549.673358,553.772090,516.828955,520.160075,0,1,c,1c
1,899.396194,802.321493,832.213881,756.311045,787.695075,939.765448,842.374552,855.194478,791.442463,804.865075,...,650.174701,637.393358,571.000746,575.247687,523.174328,527.295000,1,1,a,1a
2,886.128731,796.231269,826.007537,754.213284,787.267015,925.798806,834.662015,847.439328,788.419104,801.835672,...,645.888284,623.264478,562.774925,566.941866,520.077090,523.455075,2,1,b,1b
3,839.244478,786.159104,815.593433,750.866567,781.831045,873.725299,821.495746,834.118134,783.993209,796.509104,...,641.877239,571.947313,548.592015,552.661119,515.582388,518.900672,3,2,c,2c
4,897.840448,800.712015,830.592239,754.873507,786.201119,936.992090,840.330970,853.139776,789.753582,803.175597,...,649.052836,634.176269,568.746194,572.981269,521.431119,525.549701,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,547.927015,564.336866,582.532164,561.744254,594.801716,555.692761,573.196418,581.209552,564.646716,578.068731,...,476.103881,366.161194,361.546045,364.215299,352.793209,356.911269,4357,1453,a,1453a
4358,548.133731,564.335522,582.527687,561.743209,594.801642,555.633284,573.216716,581.228881,564.645522,570.552313,...,476.103433,365.634851,361.540522,364.219851,352.795448,354.269851,4358,1453,b,1453b
4359,469.337537,474.175597,488.778134,474.471791,507.528955,416.106343,442.375448,446.749851,439.323060,452.745373,...,364.011567,249.795299,253.101269,254.547761,249.760597,253.875448,4359,1454,c,1454c
4360,469.150075,474.172985,488.775373,474.471194,507.528881,415.144030,442.378060,446.726940,439.323060,441.592836,...,356.481194,249.533582,253.090821,254.549104,249.761567,250.074478,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,30.608636,7.721820,0.901360,100.484125,7.462002,10.482136,19.435519,3.371752,120.792794,13.806139,...,5.249208,26.265717,21.309878,98.888369,53.500107,0,1,c,1c,655.128572
1,32.765778,7.870561,0.918604,101.049217,7.505950,11.259560,19.904185,3.452418,121.743523,13.928568,...,5.838548,27.284830,22.136287,100.102472,54.233957,1,1,a,1a,667.230627
2,32.282433,7.810817,0.911753,100.768939,7.501871,11.092222,19.721948,3.421110,121.278455,13.876143,...,5.709127,26.891765,21.816668,99.509856,53.839009,2,1,b,1b,662.710162
3,30.574399,7.712012,0.900258,100.321791,7.450071,10.468317,19.410847,3.367332,120.597643,13.783964,...,5.239059,26.214046,21.267126,98.649855,53.370574,3,2,c,2c,654.061429
4,32.709101,7.854772,0.916814,100.857150,7.491714,11.226332,19.855898,3.444123,121.483731,13.899331,...,5.809079,27.177098,22.049072,99.768932,54.054448,4,2,a,2a,665.570221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,19.961453,5.535995,0.643003,75.053534,5.667868,6.657891,13.543865,2.346341,86.856700,10.003751,...,3.354051,17.276198,14.015483,67.502303,36.709452,4357,1453,a,1453a,463.525883
4358,19.968984,5.535982,0.642998,75.053395,5.667867,6.657178,13.544345,2.346419,86.856516,9.873676,...,3.349229,17.275935,14.015658,67.502732,36.437774,4358,1453,b,1453b,463.123398
4359,17.098371,4.651537,0.539517,63.393234,4.836246,4.985472,10.452741,1.803528,67.578806,7.834972,...,2.288135,12.094249,9.795332,47.788379,26.111892,4359,1454,c,1454c,355.851391
4360,17.091541,4.651511,0.539514,63.393154,4.836245,4.973942,10.452802,1.803435,67.578806,7.641972,...,2.285737,12.093749,9.795384,47.788565,25.720950,4360,1454,a,1454a,355.034394


before filling na with -1, 1514 parcels have no logsum_cml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml
0,229116,725b,725.0,Tri Valley,387.967578,387.967578
1,244166,715a,715.0,Tri Valley,387.957716,387.957716
2,202378,820a,820.0,S Leandro Hayward,468.970562,468.970562
3,2004420,1401c,1401.0,Northern Sonoma,265.713712,265.713712
4,340332,763a,763.0,Fremont,465.381779,465.381779
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,410.250920,410.250920
1956204,594355,1111a,1111.0,Martinez Concord,505.709552,505.709552
1956205,804156,1411b,1411.0,Northern Marin,442.598400,442.598400
1956206,646401,1176a,1176.0,S Ramon Danville,421.648782,421.648782


In [10]:
# third, calculate nonMandatory logsums by parcel
display(nonMandatory_NP_2050)

nonMandatory_NP_2050_scale = cnml_step1(nonMandatory_NP_2050, accessMarket_NP_2050)
display(nonMandatory_NP_2050_scale)

nonMandatory_NP_2050_sum = cnml_step2(nonMandatory_NP_2050_scale, accessMarket_NP_2050)
display(nonMandatory_NP_2050_sum)

parcels_cnml_NP_2050 = cnml_step3(parcel_subzone_lookup, nonMandatory_NP_2050_sum)
parcels_cnml_NP_2050

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.425774,11.194633,11.154072,10.876143,10.822129,10.836000,11.326673,11.312451,10.954012,10.928719,...,10.969406,11.266542,11.467966,11.462147,11.010131,11.002279,0,1,c,1c
1,10.419851,11.194356,11.154123,10.876120,10.822151,10.825585,11.329698,11.311463,10.954594,10.928308,...,10.968203,11.237525,11.467063,11.467386,11.011048,11.002083,1,1,a,1a
2,10.424597,11.195220,11.154542,10.876191,10.821888,10.847476,11.326707,11.312620,10.953252,10.928205,...,10.968943,11.244934,11.470767,11.469583,11.011047,11.001340,2,1,b,1b
3,10.499927,11.259699,11.221961,10.936040,10.886035,10.890744,11.387387,11.370873,11.008576,10.986080,...,11.023479,11.283385,11.519390,11.514401,11.062650,11.054206,3,2,c,2c
4,10.498130,11.259805,11.221949,10.936305,10.886069,10.887568,11.386025,11.369692,11.009081,10.986427,...,11.022156,11.286674,11.514200,11.520623,11.061873,11.053758,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,4.401255,6.204695,5.736764,6.051682,5.559925,5.388354,6.905456,6.693057,6.731885,6.496787,...,6.797895,6.598069,7.337373,7.279311,7.097000,7.021472,4357,1453,a,1453a
4358,4.398034,6.204639,5.736756,6.051676,5.559903,5.388744,6.907419,6.690124,6.731676,6.496644,...,6.798304,6.536254,7.342542,7.278573,7.097014,7.020651,4358,1453,b,1453b
4359,1.981415,3.728022,3.177493,3.590265,3.010090,2.961389,4.495592,4.201656,4.343443,4.027880,...,4.379952,4.004719,4.954208,4.858005,4.750990,4.644650,4359,1454,c,1454c
4360,1.991709,3.728326,3.176629,3.590302,3.009922,2.896291,4.496582,4.196972,4.343585,4.028093,...,4.378543,3.973659,4.950087,4.857190,4.751083,4.644232,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,587.540743,542.943429,554.468400,532.622400,544.019086,573.785314,534.730400,539.958286,521.851314,526.818686,...,522.144229,551.423486,531.618800,529.703771,516.298400,518.068114,0,1,c,1c
1,587.202286,542.927600,554.471314,532.621086,544.020343,573.190171,534.903257,539.901829,521.884571,526.795200,...,522.075486,549.765371,531.567200,530.003143,516.350800,518.056914,1,1,a,1a
2,587.473486,542.976971,554.495257,532.625143,544.005314,574.441086,534.732343,539.967943,521.807886,526.789314,...,522.117771,550.188743,531.778857,530.128686,516.350743,518.014457,2,1,b,1b
3,591.778057,546.661486,558.347771,536.045086,547.670857,576.913543,538.199771,543.296686,524.969257,530.096457,...,525.234114,552.385943,534.557314,532.689714,519.299486,521.035371,3,2,c,2c
4,591.675371,546.667543,558.347086,536.060229,547.672800,576.732057,538.121943,543.229200,524.998114,530.116286,...,525.158514,552.573886,534.260743,533.045257,519.255086,521.009771,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,243.282514,257.804114,244.907943,256.938914,243.321714,262.491257,282.089429,275.992914,280.586914,273.565429,...,283.772171,284.653600,295.584914,290.684571,292.690914,290.593429,4357,1453,a,1453a
4358,243.098457,257.800914,244.907486,256.938571,243.320457,262.513543,282.201600,275.825314,280.574971,273.557257,...,283.795543,281.121314,295.880286,290.642400,292.691714,290.546514,4358,1453,b,1453b
4359,105.005943,116.279943,98.663886,116.286514,97.616857,123.807543,144.382914,133.627143,144.104514,132.485029,...,145.604000,136.462171,159.404057,152.324229,158.633200,154.775029,4359,1454,c,1454c
4360,105.594171,116.297314,98.614514,116.288629,97.607257,120.087657,144.439486,133.359486,144.112629,132.497200,...,145.523486,134.687314,159.168571,152.277657,158.638514,154.751143,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,21.404615,5.326131,0.612026,71.162621,5.183960,6.874662,12.634965,2.179810,80.273703,9.116845,...,5.051060,25.402994,20.383697,98.786854,53.284943,0,1,c,1c,527.087858
1,21.392285,5.325976,0.612030,71.162446,5.183972,6.867532,12.639049,2.179582,80.278819,9.116439,...,5.035872,25.400528,20.395217,98.796880,53.283791,1,1,a,1a,527.082180
2,21.402165,5.326460,0.612056,71.162988,5.183829,6.882519,12.635011,2.179849,80.267023,9.116337,...,5.039750,25.410642,20.400048,98.796869,53.279424,2,1,b,1b,527.111991
3,21.558984,5.362604,0.616308,71.619919,5.218758,6.912142,12.716941,2.193287,80.753321,9.173568,...,5.059877,25.543408,20.498600,99.361072,53.590134,3,2,c,2c,530.227434
4,21.555243,5.362664,0.616308,71.621943,5.218776,6.909968,12.715102,2.193015,80.757759,9.173912,...,5.061598,25.529237,20.512282,99.352577,53.587501,4,2,a,2a,530.212873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,8.862992,2.528990,0.270331,34.329098,2.318614,3.144972,6.665396,1.114183,43.161242,4.734178,...,2.607438,14.124297,11.185924,56.002526,29.888452,4357,1453,a,1453a,280.872492
4358,8.856286,2.528959,0.270331,34.329052,2.318602,3.145239,6.668047,1.113506,43.159405,4.734037,...,2.575082,14.138411,11.184301,56.002679,29.883627,4358,1453,b,1453b,280.850804
4359,3.825457,1.140675,0.108906,15.536810,0.930191,1.483368,3.411575,0.539452,22.166856,2.292716,...,1.249999,7.617000,5.861636,30.352360,15.919101,4359,1454,c,1454c,143.919279
4360,3.846887,1.140846,0.108851,15.537093,0.930100,1.438800,3.412912,0.538372,22.168104,2.292926,...,1.233741,7.605747,5.859844,30.353377,15.916644,4360,1454,a,1454a,143.881055


before filling na with -1, 1514 parcels have no logsum_cnml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cnml_raw,cnml
0,229116,725b,725.0,Tri Valley,404.321776,404.321776
1,244166,715a,715.0,Tri Valley,384.647352,384.647352
2,202378,820a,820.0,S Leandro Hayward,380.232674,380.232674
3,2004420,1401c,1401.0,Northern Sonoma,197.278021,197.278021
4,340332,763a,763.0,Fremont,425.499718,425.499718
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,395.578687,395.578687
1956204,594355,1111a,1111.0,Martinez Concord,389.104868,389.104868
1956205,804156,1411b,1411.0,Northern Marin,348.562233,348.562233
1956206,646401,1176a,1176.0,S Ramon Danville,362.161424,362.161424


In [11]:
# finally, calculate combo_logsum by parcel
parcel_logsum_NP_2050 = combo_logsum(parcels_cml_NP_2050, parcels_cnml_NP_2050)
parcel_logsum_NP_2050

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA
0,229116,725b,725.0,Tri Valley,387.967578,387.967578,404.321776,404.321776,792.289354,622.289354,622.289354
1,244166,715a,715.0,Tri Valley,387.957716,387.957716,384.647352,384.647352,772.605068,602.605068,602.605068
2,202378,820a,820.0,S Leandro Hayward,468.970562,468.970562,380.232674,380.232674,849.203236,679.203236,679.203236
3,2004420,1401c,1401.0,Northern Sonoma,265.713712,265.713712,197.278021,197.278021,462.991733,292.991733,292.991733
4,340332,763a,763.0,Fremont,465.381779,465.381779,425.499718,425.499718,890.881497,720.881497,720.881497
...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,410.250920,410.250920,395.578687,395.578687,805.829607,635.829607,635.829607
1956204,594355,1111a,1111.0,Martinez Concord,505.709552,505.709552,389.104868,389.104868,894.814420,724.814420,724.814420
1956205,804156,1411b,1411.0,Northern Marin,442.598400,442.598400,348.562233,348.562233,791.160633,621.160633,621.160633
1956206,646401,1176a,1176.0,S Ramon Danville,421.648782,421.648782,362.161424,362.161424,783.810206,613.810206,613.810206


## compare with BAUS output 2030 NP

In [52]:
baus_output_2030_NP = pd.read_csv(r'X:\bayarea_urbansim\runs\run310_parcel_logsums_2030.csv')
baus_output_2030_NP.columns = [x+'_baus' for x in list(baus_output_2030_NP)]

print('BAUS parcel_logsums output: \n out of {} parcels, {} have combo_logsum = 1'.format(
    baus_output_2030_NP.shape[0], baus_output_2030_NP.loc[baus_output_2030_NP.combo_logsum_baus == 1].shape[0]))

baus_output_2030_NP[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']]

BAUS parcel_logsums output: 
 out of 1956212 parcels, 10955 have combo_logsum = 1


,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
0,229116,372.881601,388.908002,591.789602
1,244166,377.302998,370.557252,577.860250
2,202378,437.245691,383.143041,650.388732
3,2004420,263.224089,193.247357,286.471445
4,340332,461.298181,420.590585,711.888766
...,...,...,...,...
1956207,782909,419.397748,355.815375,605.213123
1956208,2054504,-1.000000,-1.000000,1.000000
1956209,2054505,-1.000000,-1.000000,1.000000
1956210,2054506,-1.000000,-1.000000,1.000000


In [53]:
# compare
compare_2050_NP = parcel_logsum_NP_2050.merge(baus_output_2030_NP[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']],
                                              left_on='PARCEL_ID', right_on='parcel_id_baus', how='outer')

# 4 synthetic parcels have no calculated value
compare_2050_NP.loc[(compare_2050_NP.combo_logsum_minus170_fillNA.isnull()) \
                    | (abs(compare_2050_NP.combo_logsum_baus - compare_2050_NP.combo_logsum_minus170_fillNA) > 1e-10)]

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
0,229116.0,725b,725.0,Tri Valley,387.967578,387.967578,404.321776,404.321776,792.289354,622.289354,622.289354,229116,372.881601,388.908002,591.789602
1,244166.0,715a,715.0,Tri Valley,387.957716,387.957716,384.647352,384.647352,772.605068,602.605068,602.605068,244166,377.302998,370.557252,577.860250
2,202378.0,820a,820.0,S Leandro Hayward,468.970562,468.970562,380.232674,380.232674,849.203236,679.203236,679.203236,202378,437.245691,383.143041,650.388732
3,2004420.0,1401c,1401.0,Northern Sonoma,265.713712,265.713712,197.278021,197.278021,462.991733,292.991733,292.991733,2004420,263.224089,193.247357,286.471445
4,340332.0,763a,763.0,Fremont,465.381779,465.381779,425.499718,425.499718,890.881497,720.881497,720.881497,340332,461.298181,420.590585,711.888766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956207,782909.0,1448a,1448.0,Southern Marin,521.213847,521.213847,351.059058,351.059058,872.272905,702.272905,702.272905,782909,419.397748,355.815375,605.213123
1956208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054504,-1.000000,-1.000000,1.000000
1956209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054505,-1.000000,-1.000000,1.000000
1956210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054506,-1.000000,-1.000000,1.000000


## 3-2. Travel Model 2050 P logsum

In [14]:
# first, modify input files
accessMarket_P_2050 = modify_accessMarket(accessMarket_P_2050_raw)
mandatory_P_2050 = create_subzone(mandatory_P_2050_raw)
nonMandatory_P_2050 = create_subzone(nonMandatory_P_2050_raw)

In [15]:
# second, calculate mandatory logsums by parcel
display(mandatory_P_2050)

mandatory_P_2050_scale = cml_step1(mandatory_P_2050, accessMarket_P_2050)
display(mandatory_P_2050_scale)

mandatory_P_2050_sum = cml_step2(mandatory_P_2050_scale, accessMarket_P_2050)
display(mandatory_P_2050_sum)

parcels_cml_P_2050 = cml_step3(parcel_subzone_lookup, mandatory_P_2050_sum)
parcels_cml_P_2050

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,11.798169,10.892757,10.881445,10.299550,10.271971,12.183129,11.218027,11.213840,10.622412,10.610437,...,10.588228,12.498610,11.488238,11.483611,10.946097,10.935573,0,1,c,1c
1,12.595492,11.097425,11.092280,10.356655,10.334246,13.051507,11.483459,11.481238,10.705075,10.705074,...,10.691165,13.361280,11.774158,11.771514,11.031185,11.031246,1,1,a,1a
2,12.417171,11.015281,11.008584,10.328332,10.328282,12.864572,11.380281,11.377492,10.664638,10.664559,...,10.633898,13.171895,11.663860,11.660144,10.989650,10.979751,2,1,b,1b
3,11.791082,10.884557,10.873329,10.287788,10.259857,12.165995,11.202473,11.197603,10.604010,10.591839,...,10.577344,12.482682,11.472744,11.467707,10.928606,10.917868,3,2,c,2c
4,12.577497,11.079929,11.074647,10.341603,10.318546,13.013900,11.454781,11.452410,10.681107,10.681100,...,10.673548,13.318248,11.743326,11.740507,11.007157,11.007192,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,7.835135,7.873044,7.710922,7.712607,7.712606,7.903781,7.876670,7.810378,7.665340,7.665334,...,8.349907,9.723015,8.966206,8.942516,8.745571,8.745598,4357,1453,a,1453a
4358,7.838030,7.873027,7.710863,7.712595,7.712605,7.901400,7.877027,7.810674,7.665327,7.564833,...,8.349899,9.715819,8.966130,8.942575,8.745601,8.710263,4358,1453,b,1453b
4359,5.847699,6.302239,6.004785,6.201637,6.201635,5.998568,6.113736,5.997259,5.976214,5.976207,...,6.857801,8.164191,7.513323,7.473264,7.365703,7.365687,4359,1454,c,1454c
4360,5.847416,6.302243,6.004792,6.201634,6.201636,5.987555,6.113749,5.997037,5.976212,5.825182,...,6.757675,8.160705,7.513204,7.473251,7.365716,7.314741,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,820.828060,764.849925,792.753955,729.813582,759.211045,880.446045,825.772537,838.982761,788.382164,801.443881,...,640.190896,574.203657,550.726194,554.833582,517.898060,521.245672,0,1,c,1c
1,880.329776,780.123657,808.487910,734.075149,763.858433,945.250373,845.580896,858.937836,794.551045,808.506343,...,647.872761,638.582015,572.063507,576.318881,524.247910,528.385448,1,1,a,1a
2,867.022239,773.993507,802.241940,731.961493,763.413358,931.300000,837.881045,851.195597,791.533358,805.482836,...,643.599104,624.448806,563.832313,568.007687,521.148284,524.542537,2,1,b,1b
3,820.299179,764.237985,792.148284,728.935821,758.307015,879.167388,824.611791,837.771045,787.008881,800.055970,...,639.378657,573.015000,549.569925,553.646716,516.592761,519.924403,3,2,c,2c
4,878.986866,778.817985,807.172015,732.951866,762.686791,942.443881,843.440746,856.786493,792.762388,806.717239,...,646.558060,635.370672,569.762612,574.004925,522.454776,526.590373,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,525.079254,539.498209,556.147761,536.758134,568.213657,561.091716,576.417537,584.993060,567.705149,581.660075,...,473.152015,367.069701,362.514851,365.199627,353.679701,357.814701,4357,1453,a,1453a
4358,525.295299,539.496940,556.143358,536.757239,568.213582,560.914030,576.444179,585.015149,567.704179,574.160000,...,473.151418,366.532687,362.509179,365.204030,353.681940,355.177761,4358,1453,b,1453b
4359,376.763134,422.273955,428.824104,423.999179,455.454627,418.911642,444.855299,449.685672,441.650970,455.605821,...,361.800821,250.739552,254.090746,255.553955,250.704478,254.836269,4359,1454,c,1454c
4360,376.742015,422.274254,428.824627,423.998955,455.454701,418.089776,444.856269,449.669104,441.650821,444.335299,...,354.328731,250.479403,254.081866,255.552985,250.705448,251.034328,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,29.919480,7.523354,0.840303,97.523175,7.217961,10.599837,19.433140,3.345502,121.365016,13.845224,...,5.166934,26.522521,21.170863,99.264716,53.496633,0,1,c,1c,651.618422
1,32.088339,7.673592,0.856981,98.092638,7.262145,11.380027,19.899296,3.425075,122.314665,13.967230,...,5.746239,27.550109,21.990681,100.481782,54.229405,1,1,a,1a,663.719764
2,31.603274,7.613293,0.850360,97.810195,7.257913,11.212076,19.718093,3.394202,121.850117,13.914998,...,5.619062,27.153701,21.673550,99.887681,53.834998,2,1,b,1b,659.198991
3,29.900202,7.517334,0.839661,97.405882,7.209366,10.584443,19.405824,3.340671,121.153610,13.821247,...,5.156238,26.466836,21.125576,99.014531,53.361028,3,2,c,2c,650.525849
4,32.039389,7.660749,0.855586,97.942536,7.251006,11.346239,19.848931,3.416496,122.039316,13.936323,...,5.717342,27.439299,21.902387,100.138094,54.045172,4,2,a,2a,662.037397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,19.139329,5.306709,0.589505,71.725655,5.402113,6.755077,13.564998,2.332701,87.393586,10.048382,...,3.303053,17.458417,13.934974,67.789238,36.723340,4357,1453,a,1453a,459.264937
4358,19.147204,5.306696,0.589501,71.725536,5.402113,6.752938,13.565625,2.332789,87.393437,9.918815,...,3.298221,17.458144,13.935142,67.789667,36.452704,4358,1453,b,1453b,458.863127
4359,13.733153,4.153647,0.454545,56.657957,4.330092,5.043347,10.468906,1.793153,67.988572,7.870750,...,2.256264,12.236802,9.751208,48.052137,26.154428,4359,1454,c,1454c,345.106315
4360,13.732383,4.153650,0.454545,56.657927,4.330093,5.033453,10.468929,1.793087,67.988549,7.676048,...,2.253923,12.236374,9.751171,48.052323,25.764226,4360,1454,a,1454a,344.298812


before filling na with -1, 1514 parcels have no logsum_cml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml
0,229116,725b,725.0,Tri Valley,384.301931,384.301931
1,244166,715a,715.0,Tri Valley,384.341200,384.341200
2,202378,820a,820.0,S Leandro Hayward,465.241020,465.241020
3,2004420,1401c,1401.0,Northern Sonoma,263.289741,263.289741
4,340332,763a,763.0,Fremont,461.731839,461.731839
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,406.893778,406.893778
1956204,594355,1111a,1111.0,Martinez Concord,502.071867,502.071867
1956205,804156,1411b,1411.0,Northern Marin,438.696294,438.696294
1956206,646401,1176a,1176.0,S Ramon Danville,418.103518,418.103518


In [16]:
# third, calculate nonMandatory logsums by parcel
display(nonMandatory_P_2050)

nonMandatory_P_2050_scale = cnml_step1(nonMandatory_P_2050, accessMarket_P_2050)
display(nonMandatory_P_2050_scale)

nonMandatory_P_2050_sum = cnml_step2(nonMandatory_P_2050_scale, accessMarket_P_2050)
display(nonMandatory_P_2050_sum)

parcels_cnml_P_2050 = cnml_step3(parcel_subzone_lookup, nonMandatory_P_2050_sum)
parcels_cnml_P_2050

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.425790,11.194643,11.154084,10.876158,10.822148,10.836010,11.326653,11.312444,10.954004,10.928722,...,10.969396,11.266537,11.467934,11.462119,11.010101,11.002255,0,1,c,1c
1,10.419854,11.194366,11.154136,10.876136,10.822170,10.825587,11.329683,11.311459,10.954586,10.928311,...,10.968190,11.237511,11.467031,11.467359,11.011019,11.002060,1,1,a,1a
2,10.424606,11.195230,11.154555,10.876206,10.821907,10.847473,11.326693,11.312615,10.953243,10.928208,...,10.968932,11.244926,11.470735,11.469552,11.011018,11.001317,2,1,b,1b
3,10.500523,11.260262,11.222831,10.936698,10.887132,10.891988,11.387787,11.371428,11.008778,10.986499,...,11.023687,11.283905,11.519545,11.514599,11.062615,11.054235,3,2,c,2c
4,10.498755,11.260363,11.222820,10.936966,10.887166,10.888741,11.386419,11.370228,11.009286,10.986847,...,11.022349,11.287179,11.514330,11.520823,11.061844,11.053788,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,4.401120,6.204539,5.736552,6.051561,5.559768,5.388378,6.905430,6.692911,6.731916,6.496708,...,6.797931,6.598231,7.337501,7.279382,7.097209,7.021624,4357,1453,a,1453a
4358,4.397892,6.204483,5.736544,6.051555,5.559745,5.388748,6.907393,6.689979,6.731707,6.496567,...,6.798340,6.536463,7.342669,7.278654,7.097222,7.020804,4358,1453,b,1453b
4359,1.981414,3.728008,3.177505,3.590264,3.010141,2.961344,4.495473,4.201547,4.343348,4.027812,...,4.379857,4.004678,4.954101,4.857881,4.750907,4.644559,4359,1454,c,1454c
4360,1.991708,3.728312,3.176641,3.590301,3.009973,2.896231,4.496464,4.196862,4.343490,4.028025,...,4.378448,3.973659,4.949980,4.857067,4.751001,4.644141,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,587.541257,542.943371,554.468971,532.622571,544.020000,573.784914,534.727086,539.956514,521.848629,526.817429,...,522.141371,551.421886,531.614114,529.699657,516.293657,518.063943,0,1,c,1c
1,587.202057,542.927543,554.471943,532.621314,544.021257,573.189314,534.900229,539.900229,521.881886,526.793943,...,522.072457,549.763257,531.562514,529.999086,516.346114,518.052800,1,1,a,1a
2,587.473600,542.976914,554.495886,532.625314,544.006229,574.439943,534.729371,539.966286,521.805143,526.788057,...,522.114857,550.186971,531.774171,530.124400,516.346057,518.010343,2,1,b,1b
3,591.811714,546.693029,558.397371,536.082000,547.733371,576.983657,538.220457,543.327029,524.978571,530.118971,...,525.243714,552.414343,534.563314,532.698514,519.294457,521.034229,3,2,c,2c
4,591.710686,546.698800,558.396743,536.097314,547.735314,576.798114,538.142286,543.258457,525.007600,530.138857,...,525.167257,552.601429,534.265314,533.054171,519.250400,521.008686,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,243.274400,257.794571,244.895714,256.931314,243.312571,262.491657,282.085771,275.983200,280.586457,273.559486,...,283.771943,284.661543,295.589371,290.686114,292.699829,290.599314,4357,1453,a,1453a
4358,243.089943,257.791371,244.895257,256.930971,243.311257,262.512800,282.197943,275.815657,280.574514,273.551429,...,283.795314,281.131943,295.884686,290.644514,292.700571,290.552457,4358,1453,b,1453b
4359,105.005486,116.278514,98.664457,116.285771,97.619600,123.804000,144.373943,133.619543,144.096857,132.479714,...,145.596286,136.458514,159.395086,152.314629,158.625429,154.767029,4359,1454,c,1454c
4360,105.593714,116.295886,98.615086,116.287886,97.610000,120.083257,144.430571,133.351829,144.104971,132.491886,...,145.515771,134.686000,159.159600,152.268114,158.630800,154.743143,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,21.416091,5.340597,0.587726,71.173030,5.172100,6.907892,12.583885,2.153114,80.334348,9.100956,...,4.961934,25.602099,20.211825,98.957202,53.170085,0,1,c,1c,527.078519
1,21.403728,5.340441,0.587729,71.172862,5.172112,6.900722,12.587959,2.152890,80.339468,9.100550,...,4.947009,25.599614,20.223250,98.967257,53.168941,1,1,a,1a,527.072936
2,21.413625,5.340927,0.587754,71.173397,5.171969,6.915778,12.583938,2.153153,80.327654,9.100448,...,4.950822,25.609807,20.228032,98.967246,53.164584,2,1,b,1b,527.102843
3,21.571751,5.377480,0.591890,71.635305,5.207403,6.946403,12.666095,2.166554,80.816177,9.157991,...,4.970865,25.744130,20.326253,99.532361,53.474932,3,2,c,2c,530.233014
4,21.568069,5.377537,0.591889,71.637352,5.207422,6.944169,12.664256,2.166281,80.820646,9.158335,...,4.972548,25.729779,20.339823,99.523917,53.472311,4,2,a,2a,530.218368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,8.867440,2.535765,0.259584,34.333093,2.313218,3.160181,6.638405,1.100502,43.194001,4.725836,...,2.561508,14.235341,11.091751,56.101321,29.824871,4357,1453,a,1453a,280.880304
4358,8.860716,2.535734,0.259584,34.333047,2.313206,3.160435,6.641045,1.099834,43.192162,4.725697,...,2.529748,14.249563,11.090164,56.101463,29.820062,4358,1453,b,1453b,280.859273
4359,3.827488,1.143760,0.104582,15.538979,0.928088,1.490497,3.397593,0.532817,22.182538,2.288633,...,1.227913,7.676336,5.811891,30.403489,15.884093,4359,1454,c,1454c,143.923597
4360,3.848929,1.143930,0.104530,15.539261,0.927996,1.445702,3.398926,0.531750,22.183787,2.288844,...,1.211963,7.664996,5.810116,30.404519,15.881642,4360,1454,a,1454a,143.885326


before filling na with -1, 1514 parcels have no logsum_cnml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cnml_raw,cnml
0,229116,725b,725.0,Tri Valley,404.338945,404.338945
1,244166,715a,715.0,Tri Valley,384.671214,384.671214
2,202378,820a,820.0,S Leandro Hayward,380.205743,380.205743
3,2004420,1401c,1401.0,Northern Sonoma,197.273217,197.273217
4,340332,763a,763.0,Fremont,425.496977,425.496977
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,395.559043,395.559043
1956204,594355,1111a,1111.0,Martinez Concord,389.079241,389.079241
1956205,804156,1411b,1411.0,Northern Marin,348.513142,348.513142
1956206,646401,1176a,1176.0,S Ramon Danville,362.154251,362.154251


In [17]:
# finally, calculate combo_logsum by parcel
parcel_logsum_P_2050 = combo_logsum(parcels_cml_P_2050, parcels_cnml_P_2050)
parcel_logsum_P_2050

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA
0,229116,725b,725.0,Tri Valley,384.301931,384.301931,404.338945,404.338945,788.640876,618.640876,618.640876
1,244166,715a,715.0,Tri Valley,384.341200,384.341200,384.671214,384.671214,769.012415,599.012415,599.012415
2,202378,820a,820.0,S Leandro Hayward,465.241020,465.241020,380.205743,380.205743,845.446763,675.446763,675.446763
3,2004420,1401c,1401.0,Northern Sonoma,263.289741,263.289741,197.273217,197.273217,460.562958,290.562958,290.562958
4,340332,763a,763.0,Fremont,461.731839,461.731839,425.496977,425.496977,887.228816,717.228816,717.228816
...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,406.893778,406.893778,395.559043,395.559043,802.452821,632.452821,632.452821
1956204,594355,1111a,1111.0,Martinez Concord,502.071867,502.071867,389.079241,389.079241,891.151108,721.151108,721.151108
1956205,804156,1411b,1411.0,Northern Marin,438.696294,438.696294,348.513142,348.513142,787.209436,617.209436,617.209436
1956206,646401,1176a,1176.0,S Ramon Danville,418.103518,418.103518,362.154251,362.154251,780.257769,610.257769,610.257769


## compare with BAUS output 2030 P

In [54]:
baus_output_2030_P = pd.read_csv(r'X:\bayarea_urbansim\runs\run313_parcel_logsums_2030.csv')
baus_output_2030_P.columns = [x+'_baus' for x in list(baus_output_2030_P)]

print('BAUS parcel_logsums output: \n out of {} parcels, {} have combo_logsum = 1'.format(
    baus_output_2030_P.shape[0], baus_output_2030_P.loc[baus_output_2030_P.combo_logsum_baus == 1].shape[0]))

baus_output_2030_P[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']]

BAUS parcel_logsums output: 
 out of 1956212 parcels, 10955 have combo_logsum = 1


,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
0,229116,384.301931,404.338945,618.640876
1,244166,384.341200,384.671214,599.012415
2,202378,465.241020,380.205743,675.446763
3,2004420,263.289741,197.273217,290.562958
4,340332,461.731839,425.496977,717.228816
...,...,...,...,...
1956207,782909,517.411112,351.080916,698.492028
1956208,2054504,-1.000000,-1.000000,1.000000
1956209,2054505,-1.000000,-1.000000,1.000000
1956210,2054506,-1.000000,-1.000000,1.000000


In [55]:
# compare
compare_2050_P = parcel_logsum_P_2050.merge(baus_output_2030_P[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']],
                                            left_on='PARCEL_ID', right_on='parcel_id_baus', how='outer')

# 4 synthetic parcels have no calculated value
compare_2050_P.loc[(compare_2050_P.combo_logsum_minus170_fillNA.isnull()) \
                    | (abs(compare_2050_P.combo_logsum_baus - compare_2050_P.combo_logsum_minus170_fillNA) > 1e-10)]

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
1956208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054504,-1.0,-1.0,1.0
1956209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054505,-1.0,-1.0,1.0
1956210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054506,-1.0,-1.0,1.0
1956211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572927,-1.0,-1.0,1.0


## 3-3.  Travel Model 2035 NP logsum

In [20]:
# first, modify input files
accessMarket_NP_2035 = modify_accessMarket(accessMarket_NP_2035_raw)
mandatory_NP_2035 = create_subzone(mandatory_NP_2035_raw)
nonMandatory_NP_2035 = create_subzone(nonMandatory_NP_2035_raw)

In [21]:
# second, calculate mandatory logsums by parcel
display(mandatory_NP_2035)

mandatory_NP_2035_scale = cml_step1(mandatory_NP_2035, accessMarket_NP_2035)
display(mandatory_NP_2035_scale)

mandatory_NP_2035_sum = cml_step2(mandatory_NP_2035_scale, accessMarket_NP_2035)
display(mandatory_NP_2035_sum)

parcels_cml_NP_2035 = cml_step3(parcel_subzone_lookup, mandatory_NP_2035_sum)
parcels_cml_NP_2035

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.874994,9.982268,9.967941,9.400159,9.366022,11.528498,10.580236,10.573812,9.996845,9.979691,...,10.300961,12.051315,11.066195,11.059474,10.547177,10.532476,0,1,c,1c
1,11.679318,10.184146,10.177100,9.454869,9.426397,12.399662,10.837828,10.834093,10.074740,10.074739,...,10.402373,12.909620,11.336144,11.332035,10.623651,10.623706,1,1,a,1a
2,11.500637,10.102709,10.093733,9.427630,9.427582,12.212716,10.736712,10.732121,10.036407,10.036328,...,10.343111,12.720680,11.230438,11.225023,10.586024,10.572062,2,1,b,1b
3,10.859219,9.966069,9.951698,9.381974,9.347182,11.506579,10.560399,10.553224,9.975087,9.957614,...,10.288282,12.033848,11.049353,11.042236,10.529038,10.514119,3,2,c,2c
4,11.656095,10.160516,10.153257,9.433957,9.404593,12.362082,10.807288,10.803363,10.048400,10.048391,...,10.384094,12.866220,11.305400,11.301104,10.599740,10.599775,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,6.906007,6.957611,6.806610,6.784972,6.784970,7.446481,7.370894,7.312490,7.140830,7.140821,...,8.073453,9.353637,8.615020,8.593678,8.368613,8.368645,4357,1453,a,1453a
4358,6.908854,6.957604,6.806534,6.784960,6.784969,7.444118,7.371233,7.312787,7.140810,7.045943,...,8.073445,9.344992,8.614988,8.593745,8.368648,8.334050,4358,1453,b,1453b
4359,5.440358,5.504351,5.260118,5.391726,5.391722,5.570763,5.614439,5.499457,5.484428,5.484420,...,6.558593,7.811463,7.142891,7.102258,7.002725,7.002704,4359,1454,c,1454c
4360,5.437807,5.504326,5.260096,5.391721,5.391722,5.558571,5.614450,5.499181,5.484426,5.340121,...,6.458984,7.808129,7.142844,7.102322,7.002733,6.952393,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,748.889627,689.369179,705.464403,654.798582,671.414179,877.783433,814.700373,829.157985,777.727836,791.764701,...,666.498284,565.922910,542.510896,546.593881,511.193806,514.391119,0,1,c,1c
1,808.913806,704.434701,721.073284,658.881418,675.919776,942.795672,833.923657,848.581940,783.540896,798.857836,...,674.066343,629.975522,562.656343,566.934254,516.900821,521.199328,1,1,a,1a
2,795.579403,698.357313,714.851866,656.848657,676.008209,928.844478,826.377687,840.972090,780.680224,795.991343,...,669.643806,615.875522,554.767836,558.948284,514.092836,517.345299,2,1,b,1b
3,747.712388,688.160299,704.252239,653.441493,670.008209,876.147687,813.220000,827.621567,776.104104,790.117164,...,665.552090,564.619403,541.254030,545.307463,509.840149,513.021194,3,2,c,2c
4,807.180746,702.671269,719.293955,657.320821,674.292612,939.991194,831.644552,846.288657,781.575224,796.891567,...,672.702239,626.736716,560.362015,564.625970,515.116418,519.413433,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,452.696567,463.648507,469.544179,459.635373,478.798358,573.155299,575.197239,585.775746,564.592388,579.908731,...,500.266343,364.603657,359.587388,362.579254,348.614403,352.911194,4357,1453,a,1453a
4358,452.909030,463.647985,469.538507,459.634478,478.798284,572.978955,575.222537,585.797910,564.590896,572.828284,...,500.265746,363.958507,359.585000,362.584254,348.617015,350.329478,4358,1453,b,1453b
4359,343.319776,355.196269,354.134328,355.661791,374.824627,433.176343,444.118507,450.474776,440.980299,456.296716,...,387.217090,249.516045,249.727015,251.279254,246.682463,250.975299,4359,1454,c,1454c
4360,343.129403,355.194403,354.132687,355.661418,374.824627,432.266493,444.119328,450.454179,440.980149,445.528134,...,379.783582,249.267239,249.723507,251.284030,246.683060,247.220746,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,27.296507,7.785459,0.709883,88.771926,6.295976,10.502188,23.513270,3.907192,126.303355,14.206426,...,4.160800,25.034237,19.896163,93.080739,50.715631,0,1,c,1c,642.524870
1,29.484346,7.955603,0.725589,89.325442,6.338226,11.280023,24.068078,3.998722,127.247398,14.333697,...,4.631730,25.963851,20.636558,94.119901,51.386876,1,1,a,1a,654.131047
2,28.998317,7.886967,0.719329,89.049858,6.339055,11.113105,23.850292,3.962863,126.782824,14.282264,...,4.528063,25.599835,20.345867,93.608609,51.006894,2,1,b,1b,649.773575
3,27.253597,7.771806,0.708663,88.587944,6.282792,10.482617,23.470545,3.899952,126.039660,14.176865,...,4.151216,24.976238,19.849337,92.834258,50.580565,3,2,c,2c,641.220663
4,29.421178,7.935687,0.723799,89.113870,6.322968,11.246469,24.002300,3.987916,126.928172,14.298416,...,4.607917,25.857979,20.552536,93.794988,51.210799,4,2,a,2a,652.327666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,16.500475,5.236260,0.472485,62.313387,4.489782,6.857483,16.600911,2.760316,91.690061,10.405150,...,2.680653,16.593207,13.197982,63.477464,34.794757,4357,1453,a,1453a,451.951358
4358,16.508220,5.236254,0.472479,62.313266,4.489781,6.855373,16.601641,2.760421,91.689818,10.278108,...,2.675909,16.593097,13.198164,63.477939,34.540216,4358,1453,b,1453b,451.568902
4359,12.513767,4.011444,0.356352,48.217548,3.514801,5.182713,12.817815,2.122746,71.615401,8.187212,...,1.834501,11.523686,9.146632,44.917183,24.744538,4359,1454,c,1454c,340.368580
4360,12.506828,4.011423,0.356351,48.217498,3.514801,5.171827,12.817838,2.122649,71.615377,7.993995,...,1.832672,11.523524,9.146806,44.917292,24.374364,4360,1454,a,1454a,339.581670


before filling na with -1, 1514 parcels have no logsum_cml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml
0,229116,725b,725.0,Tri Valley,425.506335,425.506335
1,244166,715a,715.0,Tri Valley,427.265976,427.265976
2,202378,820a,820.0,S Leandro Hayward,492.454253,492.454253
3,2004420,1401c,1401.0,Northern Sonoma,246.986327,246.986327
4,340332,763a,763.0,Fremont,506.806640,506.806640
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,392.084171,392.084171
1956204,594355,1111a,1111.0,Martinez Concord,491.301859,491.301859
1956205,804156,1411b,1411.0,Northern Marin,426.056855,426.056855
1956206,646401,1176a,1176.0,S Ramon Danville,435.546679,435.546679


In [22]:
# third, calculate nonMandatory logsums by parcel
display(nonMandatory_NP_2035)

nonMandatory_NP_2035_scale = cnml_step1(nonMandatory_NP_2035, accessMarket_NP_2035)
display(nonMandatory_NP_2035_scale)

nonMandatory_NP_2035_sum = cnml_step2(nonMandatory_NP_2035_scale, accessMarket_NP_2035)
display(nonMandatory_NP_2035_sum)

parcels_cnml_NP_2035 = cnml_step3(parcel_subzone_lookup, nonMandatory_NP_2035_sum)
parcels_cnml_NP_2035

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.447604,11.220592,11.180762,10.904400,10.851390,10.857238,11.349432,11.336082,10.979622,10.955413,...,10.994486,11.280751,11.487076,11.481622,11.033330,11.025937,0,1,c,1c
1,10.441715,11.220310,11.180782,10.904376,10.851417,10.846315,11.352629,11.335163,10.980179,10.954989,...,10.993322,11.252459,11.486368,11.486732,11.034338,11.025778,1,1,a,1a
2,10.446754,11.221254,11.181159,10.904460,10.851149,10.867937,11.349551,11.336100,10.978912,10.954964,...,10.994003,11.259818,11.490003,11.488452,11.034307,11.025004,2,1,b,1b
3,10.518241,11.282125,11.245064,10.961065,10.912160,10.909587,11.407455,11.391753,11.031164,11.009597,...,11.045546,11.296790,11.535833,11.531007,11.083152,11.075126,3,2,c,2c
4,10.516030,11.282214,11.245070,10.961320,10.912195,10.907563,11.406020,11.390524,11.031664,11.009990,...,11.044356,11.300035,11.531384,11.537230,11.082253,11.074623,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,4.521860,6.299084,5.845809,6.146783,5.670603,5.488155,6.973451,6.768277,6.801059,6.573712,...,6.864059,6.675468,7.394240,7.338326,7.153287,7.080203,4357,1453,a,1453a
4358,4.519985,6.298994,5.845714,6.146777,5.670597,5.490573,6.975952,6.765408,6.800782,6.573516,...,6.864617,6.618797,7.398787,7.337382,7.153199,7.079253,4358,1453,b,1453b
4359,2.050023,3.801722,3.249471,3.664485,3.082841,3.029267,4.562673,4.271659,4.410948,4.098506,...,4.446307,4.069386,5.015447,4.920416,4.811683,4.706969,4359,1454,c,1454c
4360,2.060380,3.802029,3.248647,3.664521,3.082678,2.964887,4.563700,4.267075,4.411096,4.098713,...,4.444904,4.038502,5.010982,4.919500,4.811763,4.706504,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,578.790743,527.513200,540.843600,517.325314,528.716400,558.193200,519.469886,524.527829,506.769029,511.571943,...,507.043029,535.496914,516.559029,514.466629,501.534971,503.163771,0,1,c,1c
1,578.454229,527.497086,540.844743,517.323943,528.717943,557.569029,519.652571,524.475314,506.800857,511.547714,...,506.976514,533.880229,516.518571,514.758629,501.592571,503.154686,1,1,a,1a
2,578.742171,527.551029,540.866286,517.328743,528.702629,558.804571,519.476686,524.528857,506.728457,511.546286,...,507.015429,534.300743,516.726286,514.856914,501.590800,503.110457,2,1,b,1b
3,582.827143,531.029371,544.518000,520.563314,532.188971,561.184571,522.785486,527.709029,509.714286,514.668171,...,509.960743,536.413429,519.345143,517.288629,504.381943,505.974571,3,2,c,2c
4,582.700800,531.034457,544.518343,520.577886,532.190971,561.068914,522.703486,527.638800,509.742857,514.690629,...,509.892743,536.598857,519.090914,517.644229,504.330571,505.945829,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,240.176800,246.284171,235.989143,245.461486,232.671429,251.388457,269.413829,263.510400,267.994000,261.189029,...,271.018629,272.337886,282.682686,277.706857,279.818229,277.693257,4357,1453,a,1453a
4358,240.069657,246.279029,235.983714,245.461143,232.671086,251.526629,269.556743,263.346457,267.978171,261.177829,...,271.050514,269.099543,282.942514,277.652914,279.813200,277.638971,4358,1453,b,1453b
4359,98.928971,103.577771,87.626971,103.615886,84.799314,110.880571,131.655086,120.846514,131.416229,119.748686,...,132.861371,123.418914,146.751657,139.540571,146.012286,142.079886,4359,1454,c,1454c
4360,99.520800,103.595314,87.579886,103.617943,84.790000,107.201714,131.713771,120.584571,131.424686,119.760514,...,132.781200,121.654114,146.496514,139.488229,146.016857,142.053314,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,21.096521,5.957522,0.544231,70.134490,4.957873,6.678469,14.992550,2.471701,82.299521,9.179001,...,3.937101,23.836684,18.726723,91.322010,49.608687,0,1,c,1c,512.400012
1,21.084255,5.957340,0.544232,70.134304,4.957887,6.671002,14.997822,2.471454,82.304690,9.178567,...,3.925214,23.834817,18.737352,91.332498,49.607792,1,1,a,1a,512.397747
2,21.094750,5.957949,0.544254,70.134955,4.957743,6.685784,14.992746,2.471706,82.292932,9.178541,...,3.928306,23.844402,18.740929,91.332175,49.603431,2,1,b,1b,512.425753
3,21.243645,5.997232,0.547928,70.573470,4.990435,6.714259,15.088242,2.486692,82.777832,9.234556,...,3.943839,23.965250,18.829444,91.840401,49.885814,3,2,c,2c,515.383802
4,21.239039,5.997290,0.547929,70.575446,4.990454,6.712876,15.085875,2.486361,82.782472,9.234959,...,3.945202,23.953519,18.842388,91.831047,49.882980,4,2,a,2a,515.369794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,8.754278,2.781435,0.237467,33.277545,2.181803,3.007722,7.775620,1.241724,43.522348,4.686446,...,2.002293,13.044430,10.108604,50.950710,27.378756,4357,1453,a,1453a,268.441420
4358,8.750372,2.781376,0.237462,33.277498,2.181800,3.009375,7.779744,1.240952,43.519777,4.686245,...,1.978484,13.056420,10.106640,50.949795,27.373403,4358,1453,b,1453b,268.428412
4359,3.605892,1.169766,0.088176,14.047345,0.795179,1.326624,3.799730,0.569458,21.342055,2.148619,...,0.907405,6.771875,5.079314,26.586651,14.008156,4359,1454,c,1454c,131.198536
4360,3.627464,1.169964,0.088128,14.047624,0.795092,1.282609,3.801424,0.568224,21.343429,2.148832,...,0.894430,6.760101,5.077409,26.587484,14.005536,4360,1454,a,1454a,131.163628


before filling na with -1, 1514 parcels have no logsum_cnml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cnml_raw,cnml
0,229116,725b,725.0,Tri Valley,365.653815,365.653815
1,244166,715a,715.0,Tri Valley,348.505321,348.505321
2,202378,820a,820.0,S Leandro Hayward,359.664085,359.664085
3,2004420,1401c,1401.0,Northern Sonoma,183.338169,183.338169
4,340332,763a,763.0,Fremont,400.890624,400.890624
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,376.356424,376.356424
1956204,594355,1111a,1111.0,Martinez Concord,378.053120,378.053120
1956205,804156,1411b,1411.0,Northern Marin,330.436147,330.436147
1956206,646401,1176a,1176.0,S Ramon Danville,340.278633,340.278633


In [23]:
# finally, calculate combo_logsum by parcel
parcel_logsum_NP_2035 = combo_logsum(parcels_cml_NP_2035, parcels_cnml_NP_2035)
parcel_logsum_NP_2035

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA
0,229116,725b,725.0,Tri Valley,425.506335,425.506335,365.653815,365.653815,791.160150,621.160150,621.160150
1,244166,715a,715.0,Tri Valley,427.265976,427.265976,348.505321,348.505321,775.771297,605.771297,605.771297
2,202378,820a,820.0,S Leandro Hayward,492.454253,492.454253,359.664085,359.664085,852.118338,682.118338,682.118338
3,2004420,1401c,1401.0,Northern Sonoma,246.986327,246.986327,183.338169,183.338169,430.324496,260.324496,260.324496
4,340332,763a,763.0,Fremont,506.806640,506.806640,400.890624,400.890624,907.697264,737.697264,737.697264
...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,392.084171,392.084171,376.356424,376.356424,768.440596,598.440596,598.440596
1956204,594355,1111a,1111.0,Martinez Concord,491.301859,491.301859,378.053120,378.053120,869.354979,699.354979,699.354979
1956205,804156,1411b,1411.0,Northern Marin,426.056855,426.056855,330.436147,330.436147,756.493002,586.493002,586.493002
1956206,646401,1176a,1176.0,S Ramon Danville,435.546679,435.546679,340.278633,340.278633,775.825312,605.825312,605.825312


## compare with BAUS output 2015 NP

In [56]:
baus_output_2015_NP = pd.read_csv(r'X:\bayarea_urbansim\runs\run310_parcel_logsums_2015.csv')
baus_output_2015_NP.columns = [x+'_baus' for x in list(baus_output_2015_NP)]

print('BAUS parcel_logsums output: \n out of {} parcels, {} have combo_logsum = 1'.format(
    baus_output_2015_NP.shape[0], baus_output_2015_NP.loc[baus_output_2015_NP.combo_logsum_baus == 1].shape[0]))

baus_output_2015_NP[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']]

BAUS parcel_logsums output: 
 out of 1956212 parcels, 10955 have combo_logsum = 1


,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
0,229116,425.506335,365.653815,621.160150
1,244166,427.265976,348.505321,605.771297
2,202378,492.454253,359.664085,682.118338
3,2004420,246.986327,183.338169,260.324496
4,340332,506.806640,400.890624,737.697264
...,...,...,...,...
1956207,782909,508.162177,337.219823,675.382000
1956208,2054504,-1.000000,-1.000000,1.000000
1956209,2054505,-1.000000,-1.000000,1.000000
1956210,2054506,-1.000000,-1.000000,1.000000


In [25]:
# compare
compare_2035_NP = parcel_logsum_NP_2035.merge(baus_output_2015_NP[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']],
                                              left_on='PARCEL_ID', right_on='parcel_id_baus', how='outer')

# 4 synthetic parcels have no calculated value
compare_2035_NP.loc[(compare_2035_NP.combo_logsum_minus170_fillNA.isnull()) \
                    | (abs(compare_2035_NP.combo_logsum_baus - compare_2035_NP.combo_logsum_minus170_fillNA) > 1e-10)]

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
1956208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054504,-1.0,-1.0,1.0
1956209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054505,-1.0,-1.0,1.0
1956210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054506,-1.0,-1.0,1.0
1956211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572927,-1.0,-1.0,1.0


## 3-4. Travel Model 2035 P logsum

In [26]:
# first, modify input files
accessMarket_P_2035 = modify_accessMarket(accessMarket_P_2035_raw)
mandatory_P_2035 = create_subzone(mandatory_P_2035_raw)
nonMandatory_P_2035 = create_subzone(nonMandatory_P_2035_raw)

In [27]:
# second, calculate mandatory logsums by parcel
display(mandatory_P_2035)

mandatory_P_2035_scale = cml_step1(mandatory_P_2035, accessMarket_P_2035)
display(mandatory_P_2035_scale)

mandatory_P_2035_sum = cml_step2(mandatory_P_2035_scale, accessMarket_P_2035)
display(mandatory_P_2035_sum)

parcels_cml_P_2035 = cml_step3(parcel_subzone_lookup, mandatory_P_2035_sum)
parcels_cml_P_2035

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.863789,9.971092,9.956762,9.388248,9.354083,11.531064,10.582744,10.576354,9.999096,9.981990,...,10.294583,12.048954,11.063899,11.057199,10.544896,10.530228,0,1,c,1c
1,11.665375,10.171684,10.164641,9.442629,9.414120,12.403090,10.840493,10.836778,10.077084,10.077085,...,10.395881,12.907209,11.333708,11.329615,10.621301,10.621356,1,1,a,1a
2,11.487089,10.090672,10.081695,9.415544,9.415494,12.215918,10.739241,10.734675,10.038696,10.038618,...,10.336651,12.718244,11.228022,11.222627,10.583701,10.569770,2,1,b,1b
3,10.849709,9.956451,9.942095,9.371248,9.336445,11.508298,10.562145,10.554991,9.976713,9.959245,...,10.282276,12.031184,11.046944,11.039836,10.526684,10.511758,3,2,c,2c
4,11.645450,10.150057,10.142817,9.423101,9.393731,12.363916,10.808986,10.805068,10.050021,10.050014,...,10.377969,12.863289,11.302630,11.298335,10.597252,10.597286,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,6.899649,6.946764,6.796930,6.773280,6.773279,7.451190,7.373363,7.314981,7.143657,7.143648,...,8.067755,9.354621,8.614947,8.593578,8.368837,8.368869,4357,1453,a,1453a
4358,6.902773,6.946752,6.796873,6.773267,6.773277,7.448736,7.373756,7.315299,7.143639,7.048727,...,8.067748,9.345928,8.614898,8.593631,8.368871,8.334239,4358,1453,b,1453b
4359,5.368240,5.466592,5.217440,5.355171,5.355168,5.508290,5.593247,5.475760,5.464140,5.464135,...,6.557475,7.805532,7.138393,7.097549,6.998960,6.998943,4359,1454,c,1454c
4360,5.365399,5.466543,5.217391,5.355168,5.355167,5.496053,5.593273,5.475503,5.464139,5.316915,...,6.458169,7.801675,7.138330,7.097599,6.998970,6.948411,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,744.414104,684.226791,699.257388,649.780522,665.327910,879.376791,816.247164,830.738284,779.252388,793.324627,...,664.321567,568.410970,543.925075,548.070746,512.489254,515.774030,0,1,c,1c
1,804.233955,699.196343,714.770746,653.838806,669.808284,944.453358,835.482164,850.172910,785.072388,800.421269,...,671.881119,632.459851,564.060075,568.400299,518.191119,522.574627,1,1,a,1a
2,790.929030,693.150672,708.580746,651.817537,669.910821,930.485299,827.926045,842.553284,782.207612,797.550597,...,667.460970,618.357985,556.173060,560.416119,515.385149,518.724925,2,1,b,1b
3,743.363358,683.134179,698.162836,648.511866,664.011642,877.677836,814.709925,829.144030,777.582015,791.627239,...,663.403134,567.084851,542.659776,546.775000,511.130149,514.395672,3,2,c,2c
4,802.747015,697.582388,713.142090,652.381493,668.286716,941.529925,833.130896,847.806493,783.052761,798.401045,...,670.544403,629.182239,561.740821,566.065970,516.396418,520.778358,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,448.582761,458.530672,463.449030,454.633657,472.730597,574.908582,576.741119,587.352239,566.159925,581.508060,...,498.140373,367.341343,361.167463,364.218433,350.096791,354.478582,4357,1453,a,1453a
4358,448.815896,458.529776,463.444776,454.632687,472.730448,574.725448,576.770448,587.375970,566.158582,574.424403,...,498.139851,366.692612,361.163806,364.222388,350.099328,351.894254,4358,1453,b,1453b
4359,334.298507,348.070075,345.576642,348.804627,366.901418,429.916045,443.896642,450.096940,440.822836,456.171269,...,385.432910,251.737687,250.976866,252.574478,247.867164,252.245299,4359,1454,c,1454c
4360,334.086493,348.066418,345.572985,348.804403,366.901343,429.002836,443.898582,450.077761,440.822761,445.184701,...,378.022015,251.449851,250.972164,252.578209,247.867910,248.474254,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,27.264307,7.775401,0.751339,87.883301,6.243109,10.556900,23.550085,3.922424,126.457872,14.297180,...,4.171040,25.039509,20.190017,93.406536,50.599725,0,1,c,1c,642.015734
1,29.455221,7.945512,0.768007,88.432187,6.285151,11.338143,24.105047,4.014187,127.402348,14.425074,...,4.641035,25.966421,20.938924,94.445760,51.266894,1,1,a,1a,653.624132
2,28.967925,7.876810,0.761356,88.158809,6.286113,11.170456,23.887040,3.978210,126.937449,14.373339,...,4.537555,25.603344,20.644800,93.934343,50.889221,2,1,b,1b,649.266016
3,27.225824,7.762985,0.750162,87.711715,6.230758,10.536504,23.505733,3.914897,126.186803,14.266590,...,4.161309,24.981262,20.142284,93.158824,50.464502,3,2,c,2c,640.723451
4,29.400761,7.927171,0.766257,88.235084,6.270873,11.303047,24.037209,4.003014,127.074601,14.388666,...,4.616984,25.859655,20.852932,94.118657,51.090672,4,2,a,2a,651.832377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,16.429428,5.210641,0.497967,61.489542,4.435871,6.901765,16.639938,2.773250,91.877010,10.479853,...,2.695577,16.626290,13.417203,63.808808,34.775925,4357,1453,a,1453a,451.508495
4358,16.437967,5.210630,0.497962,61.489411,4.435870,6.899567,16.640784,2.773362,91.876792,10.352192,...,2.690816,16.626122,13.417349,63.809270,34.522391,4358,1453,b,1453b,451.127075
4359,12.243746,3.955391,0.371315,47.176086,3.442822,5.161133,12.807155,2.125183,71.537179,8.221051,...,1.847269,11.553682,9.304425,45.176387,24.746385,4359,1454,c,1454c,338.976674
4360,12.235981,3.955349,0.371311,47.176056,3.442821,5.150170,12.807211,2.125093,71.537167,8.023053,...,1.845157,11.553466,9.304562,45.176523,24.376429,4360,1454,a,1454a,338.185961


before filling na with -1, 1514 parcels have no logsum_cml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml
0,229116,725b,725.0,Tri Valley,424.787944,424.787944
1,244166,715a,715.0,Tri Valley,425.969991,425.969991
2,202378,820a,820.0,S Leandro Hayward,492.116127,492.116127
3,2004420,1401c,1401.0,Northern Sonoma,246.652741,246.652741
4,340332,763a,763.0,Fremont,506.431033,506.431033
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,391.330732,391.330732
1956204,594355,1111a,1111.0,Martinez Concord,490.533255,490.533255
1956205,804156,1411b,1411.0,Northern Marin,425.743963,425.743963
1956206,646401,1176a,1176.0,S Ramon Danville,434.774323,434.774323


In [28]:
# third, calculate nonMandatory logsums by parcel
display(nonMandatory_P_2035)

nonMandatory_P_2035_scale = cnml_step1(nonMandatory_P_2035, accessMarket_P_2035)
display(nonMandatory_P_2035_scale)

nonMandatory_P_2035_sum = cnml_step2(nonMandatory_P_2035_scale, accessMarket_P_2035)
display(nonMandatory_P_2035_sum)

parcels_cnml_P_2035 = cnml_step3(parcel_subzone_lookup, nonMandatory_P_2035_sum)
parcels_cnml_P_2035

,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,10.447568,11.220421,11.180576,10.904193,10.851150,10.857234,11.349336,11.335921,10.979561,10.955266,...,10.994422,11.280698,11.487003,11.481514,11.033380,11.025945,0,1,c,1c
1,10.441690,11.220138,11.180596,10.904170,10.851176,10.846327,11.352530,11.334999,10.980117,10.954844,...,10.993258,11.252412,11.486288,11.486630,11.034389,11.025787,1,1,a,1a
2,10.446726,11.221084,11.180973,10.904254,10.850909,10.867958,11.349457,11.335934,10.978851,10.954816,...,10.993938,11.259765,11.489927,11.488353,11.034359,11.025015,2,1,b,1b
3,10.518153,11.281908,11.244845,10.960832,10.911922,10.909396,11.407246,11.391501,11.031012,11.009389,...,11.045387,11.296559,11.535629,11.530783,11.083072,11.075018,3,2,c,2c
4,10.515936,11.281997,11.244851,10.961085,10.911957,10.907379,11.405811,11.390272,11.031511,11.009782,...,11.044200,11.299812,11.531186,11.537004,11.082172,11.074514,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,4.524647,6.301928,5.851542,6.149744,5.676724,5.489524,6.973969,6.769500,6.801559,6.574975,...,6.864570,6.676847,7.394466,7.338683,7.153332,7.080372,4357,1453,a,1453a
4358,4.522936,6.301834,5.851452,6.149739,5.676718,5.492273,6.976453,6.766590,6.801283,6.574779,...,6.865129,6.619726,7.399021,7.337747,7.153251,7.079424,4358,1453,b,1453b
4359,2.049839,3.801541,3.249228,3.664315,3.082628,3.029043,4.562595,4.271477,4.410912,4.098391,...,4.446280,4.069314,5.015478,4.920399,4.811788,4.707030,4359,1454,c,1454c
4360,2.060188,3.801848,3.248405,3.664351,3.082466,2.964710,4.563621,4.266895,4.411060,4.098598,...,4.444876,4.038403,5.011015,4.919486,4.811866,4.706565,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,highInc_autos_ge_workers_AV,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub
0,578.765086,527.502629,540.813543,517.312686,528.702571,558.191600,519.461200,524.516743,506.762229,511.561600,...,507.036000,535.492229,516.550457,514.456629,501.533086,503.159943,0,1,c,1c
1,578.429200,527.486457,540.814686,517.311371,528.704057,557.568343,519.643714,524.464057,506.794000,511.537486,...,506.969486,533.875886,516.509600,514.748971,501.590743,503.150914,1,1,a,1a
2,578.716971,527.540514,540.836229,517.316171,528.688800,558.804400,519.468114,524.517486,506.721657,511.535886,...,507.008343,534.296057,516.717543,514.847429,501.589029,503.106800,2,1,b,1b
3,582.798514,531.016171,544.486057,520.549200,532.175257,561.172286,522.770343,527.692743,509.702286,514.654343,...,509.948286,536.398571,519.329086,517.272000,504.372629,505.964114,3,2,c,2c
4,582.671829,531.021257,544.486400,520.563657,532.177257,561.057029,522.688343,527.622514,509.730800,514.676800,...,509.880457,536.584457,519.075200,517.627486,504.321200,505.935314,4,2,a,2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,240.312457,246.445886,236.297314,245.629886,233.021086,251.465314,269.440229,263.578400,268.019257,261.259257,...,271.044457,272.415029,282.691200,277.723429,279.816057,277.698629,4357,1453,a,1453a
4358,240.214686,246.440514,236.292171,245.629600,233.020743,251.622400,269.582171,263.412114,268.003486,261.248057,...,271.076400,269.150971,282.951486,277.669943,279.811429,277.644457,4358,1453,b,1453b
4359,98.894857,103.566629,87.593657,103.605371,84.787029,110.866400,131.647429,120.834229,131.410857,119.740171,...,132.856457,123.413143,146.749029,139.535771,146.013543,142.079086,4359,1454,c,1454c
4360,99.486229,103.584171,87.546629,103.607429,84.777771,107.190229,131.706057,120.572400,131.419314,119.752000,...,132.776229,121.646800,146.494000,139.483600,146.018000,142.052514,4360,1454,a,1454a


,lowInc_0_autos_noAV,lowInc_autos_lt_workers_noAV,lowInc_autos_lt_workers_AV,lowInc_autos_ge_workers_noAV,lowInc_autos_ge_workers_AV,medInc_0_autos_noAV,medInc_autos_lt_workers_noAV,medInc_autos_lt_workers_AV,medInc_autos_ge_workers_noAV,medInc_autos_ge_workers_AV,...,veryHighInc_0_autos_noAV,veryHighInc_autos_lt_workers_noAV,veryHighInc_autos_lt_workers_AV,veryHighInc_autos_ge_workers_noAV,veryHighInc_autos_ge_workers_AV,destChoiceAlt,taz,subzone,taz_sub,weighted_sum
0,21.197381,5.994423,0.581094,69.966927,4.961085,6.701078,14.987317,2.476565,82.237891,9.219288,...,3.929480,23.779323,18.951729,91.409659,49.362227,0,1,c,1c,512.411596
1,21.185079,5.994239,0.581095,69.966750,4.961098,6.693596,14.992583,2.476316,82.243047,9.218853,...,3.917619,23.777442,18.962498,91.420168,49.361342,1,1,a,1a,512.409470
2,21.195619,5.994853,0.581118,69.967399,4.960955,6.708435,14.987517,2.476568,82.231307,9.218824,...,3.920702,23.787015,18.966125,91.419855,49.357014,2,1,b,1b,512.437680
3,21.345106,6.034350,0.585040,70.404668,4.993670,6.736861,15.082792,2.491561,82.715007,9.275025,...,3.936131,23.907237,19.055443,91.927196,49.637329,3,2,c,2c,515.390009
4,21.340466,6.034407,0.585040,70.406624,4.993689,6.735478,15.080426,2.491229,82.719634,9.275429,...,3.937495,23.895549,19.068538,91.917822,49.634504,4,2,a,2a,515.376167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4357,8.801489,2.800556,0.253897,33.221626,2.186555,3.018836,7.773798,1.244515,43.494438,4.708376,...,1.999001,13.013647,10.230871,50.999408,27.243470,4357,1453,a,1453a,268.485149
4358,8.797908,2.800495,0.253891,33.221587,2.186552,3.020722,7.777893,1.243730,43.491879,4.708174,...,1.975049,13.025629,10.228900,50.998564,27.238156,4358,1453,b,1453b,268.472706
4359,3.622043,1.176908,0.094118,14.012704,0.795600,1.330949,3.798247,0.570532,21.325488,2.157940,...,0.905614,6.755570,5.140266,26.612498,13.938590,4359,1454,c,1454c,131.190883
4360,3.643702,1.177107,0.094067,14.012982,0.795513,1.286816,3.799938,0.569296,21.326860,2.158153,...,0.892653,6.743829,5.138345,26.613310,13.935983,4360,1454,a,1454a,131.155951


before filling na with -1, 1514 parcels have no logsum_cnml


,PARCEL_ID,taz_sub,TAZ,SD_NAME,cnml_raw,cnml
0,229116,725b,725.0,Tri Valley,365.643917,365.643917
1,244166,715a,715.0,Tri Valley,348.539250,348.539250
2,202378,820a,820.0,S Leandro Hayward,359.639886,359.639886
3,2004420,1401c,1401.0,Northern Sonoma,183.330399,183.330399
4,340332,763a,763.0,Fremont,400.876065,400.876065
...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,376.346688,376.346688
1956204,594355,1111a,1111.0,Martinez Concord,378.056170,378.056170
1956205,804156,1411b,1411.0,Northern Marin,330.469209,330.469209
1956206,646401,1176a,1176.0,S Ramon Danville,340.260615,340.260615


In [29]:
# finally, calculate combo_logsum by parcel
parcel_logsum_P_2035 = combo_logsum(parcels_cml_P_2035, parcels_cnml_P_2035)
parcel_logsum_P_2035

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA
0,229116,725b,725.0,Tri Valley,424.787944,424.787944,365.643917,365.643917,790.431861,620.431861,620.431861
1,244166,715a,715.0,Tri Valley,425.969991,425.969991,348.539250,348.539250,774.509242,604.509242,604.509242
2,202378,820a,820.0,S Leandro Hayward,492.116127,492.116127,359.639886,359.639886,851.756013,681.756013,681.756013
3,2004420,1401c,1401.0,Northern Sonoma,246.652741,246.652741,183.330399,183.330399,429.983140,259.983140,259.983140
4,340332,763a,763.0,Fremont,506.431033,506.431033,400.876065,400.876065,907.307097,737.307097,737.307097
...,...,...,...,...,...,...,...,...,...,...,...
1956203,580747,1193a,1193.0,East Contra Costa,391.330732,391.330732,376.346688,376.346688,767.677420,597.677420,597.677420
1956204,594355,1111a,1111.0,Martinez Concord,490.533255,490.533255,378.056170,378.056170,868.589425,698.589425,698.589425
1956205,804156,1411b,1411.0,Northern Marin,425.743963,425.743963,330.469209,330.469209,756.213172,586.213172,586.213172
1956206,646401,1176a,1176.0,S Ramon Danville,434.774323,434.774323,340.260615,340.260615,775.034938,605.034938,605.034938


## compare with BAUS output 2015 P

In [57]:
baus_output_2015_P = pd.read_csv(r'X:\bayarea_urbansim\runs\run313_parcel_logsums_2015.csv')
baus_output_2015_P.columns = [x+'_baus' for x in list(baus_output_2015_P)]

print('BAUS parcel_logsums output: \n out of {} parcels, {} have combo_logsum = 1'.format(
    baus_output_2015_P.shape[0], baus_output_2015_P.loc[baus_output_2015_P.combo_logsum_baus == 1].shape[0]))

baus_output_2015_P[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']]

BAUS parcel_logsums output: 
 out of 1956212 parcels, 10955 have combo_logsum = 1


,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
0,229116,424.787944,365.643917,620.431861
1,244166,425.969991,348.539250,604.509242
2,202378,492.116127,359.639886,681.756013
3,2004420,246.652741,183.330399,259.983140
4,340332,506.431033,400.876065,737.307097
...,...,...,...,...
1956207,782909,507.751750,337.242654,674.994403
1956208,2054504,-1.000000,-1.000000,1.000000
1956209,2054505,-1.000000,-1.000000,1.000000
1956210,2054506,-1.000000,-1.000000,1.000000


In [31]:
# compare
compare_2035_P = parcel_logsum_P_2035.merge(baus_output_2015_P[['parcel_id_baus', 'cml_baus', 'cnml_baus', 'combo_logsum_baus']],
                                            left_on='PARCEL_ID', right_on='parcel_id_baus', how='outer')

# 4 synthetic parcels have no calculated value
compare_2035_P.loc[(compare_2035_P.combo_logsum_minus170_fillNA.isnull()) \
                    | (abs(compare_2035_P.combo_logsum_baus - compare_2035_P.combo_logsum_minus170_fillNA) > 1e-10)]

,PARCEL_ID,taz_sub,TAZ,SD_NAME,cml_raw,cml,cnml_raw,cnml,combo_logsum_raw,combo_logsum_minus170,combo_logsum_minus170_fillNA,parcel_id_baus,cml_baus,cnml_baus,combo_logsum_baus
1956208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054504,-1.0,-1.0,1.0
1956209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054505,-1.0,-1.0,1.0
1956210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2054506,-1.0,-1.0,1.0
1956211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572927,-1.0,-1.0,1.0


# 4. compare NP vs P calculated combo_logsum (not BAUS output)

## 4-1. TM 2035 inputs, BAUS 2010-2025

In [32]:
# add "_np", '_p' to field names
parcel_logsum_NP_2035_tag = parcel_logsum_NP_2035[['PARCEL_ID', 'SD_NAME',
                                                 'cml_raw', 'cml', 'cnml_raw', 'cnml',
                                                 'combo_logsum_raw', 'combo_logsum_minus170',
                                                 'combo_logsum_minus170_fillNA']]
parcel_logsum_NP_2035_tag.columns = ['PARCEL_ID', 'SD_NAME'] + [x+'_NP' for x in list(parcel_logsum_NP_2035_tag)[2:]]


parcel_logsum_P_2035_tag = parcel_logsum_P_2035[['PARCEL_ID', 'SD_NAME',
                                                 'cml_raw', 'cml', 'cnml_raw', 'cnml',
                                                 'combo_logsum_raw', 'combo_logsum_minus170',
                                                 'combo_logsum_minus170_fillNA']]
parcel_logsum_P_2035_tag.columns = ['PARCEL_ID', 'SD_NAME'] + [x+'_P' for x in list(parcel_logsum_P_2035_tag)[2:]]


parcel_combo_logsum_compare_2035 = parcel_logsum_NP_2035_tag.merge(parcel_logsum_P_2035_tag,
                                                                   on=['PARCEL_ID', 'SD_NAME'],
                                                                   how='outer')

parcel_combo_logsum_compare_2035['diff'] = \
    parcel_combo_logsum_compare_2035['combo_logsum_minus170_fillNA_P'] - parcel_combo_logsum_compare_2035['combo_logsum_minus170_fillNA_NP']

parcel_combo_logsum_compare_2035['diff_tag'] = 'P > NP'
parcel_combo_logsum_compare_2035.loc[parcel_combo_logsum_compare_2035['diff'] < 0, 'diff_tag'] = 'P < NP'
parcel_combo_logsum_compare_2035.loc[parcel_combo_logsum_compare_2035['diff'] == 0, 'diff_tag'] = 'P = NP'

In [33]:
# count how many parcels have combo_logsum NP < P or NP > P

SD_stats_2035 = parcel_combo_logsum_compare_2035.groupby(['SD_NAME', 'diff_tag'])['PARCEL_ID'].count().reset_index()

# project area
SD_stats_2035.loc[SD_stats_2035.SD_NAME.isin(['San Jose CBD', 'West San Jose', 'East San Jose',
                                                   'South San Jose', 'SE Snta Clara Cnty'])]

,SD_NAME,diff_tag,PARCEL_ID
11,East San Jose,P < NP,68475
12,East San Jose,P > NP,22263
44,SE Snta Clara Cnty,P < NP,20
45,SE Snta Clara Cnty,P = NP,6
46,SE Snta Clara Cnty,P > NP,34011
53,San Jose CBD,P < NP,47941
54,San Jose CBD,P > NP,16021
63,South San Jose,P > NP,59623
73,West San Jose,P < NP,36234
74,West San Jose,P = NP,8


## 4-2. TM 2050 inputs, BAUS 2030-2050

In [34]:
# add "_np", '_p' to field names
parcel_logsum_NP_2050_tag = parcel_logsum_NP_2050[['PARCEL_ID', 'SD_NAME',
                                                 'cml_raw', 'cml', 'cnml_raw', 'cnml',
                                                 'combo_logsum_raw', 'combo_logsum_minus170',
                                                 'combo_logsum_minus170_fillNA']]
parcel_logsum_NP_2050_tag.columns = ['PARCEL_ID', 'SD_NAME'] + [x+'_NP' for x in list(parcel_logsum_NP_2050_tag)[2:]]


parcel_logsum_P_2050_tag = parcel_logsum_P_2050[['PARCEL_ID', 'SD_NAME',
                                                 'cml_raw', 'cml', 'cnml_raw', 'cnml',
                                                 'combo_logsum_raw', 'combo_logsum_minus170',
                                                 'combo_logsum_minus170_fillNA']]
parcel_logsum_P_2050_tag.columns = ['PARCEL_ID', 'SD_NAME'] + [x+'_P' for x in list(parcel_logsum_P_2050_tag)[2:]]


parcel_combo_logsum_compare_2050 = parcel_logsum_NP_2050_tag.merge(parcel_logsum_P_2050_tag,
                                                                   on=['PARCEL_ID', 'SD_NAME'],
                                                                   how='outer')

parcel_combo_logsum_compare_2050['diff'] = \
    parcel_combo_logsum_compare_2050['combo_logsum_minus170_fillNA_P'] - parcel_combo_logsum_compare_2050['combo_logsum_minus170_fillNA_NP']

parcel_combo_logsum_compare_2050['diff_tag'] = 'P > NP'
parcel_combo_logsum_compare_2050.loc[parcel_combo_logsum_compare_2050['diff'] < 0, 'diff_tag'] = 'P < NP'
parcel_combo_logsum_compare_2050.loc[parcel_combo_logsum_compare_2050['diff'] == 0, 'diff_tag'] = 'P = NP'

In [35]:
# count how many parcels have combo_logsum NP < P or NP > P

SD_stats_2050 = parcel_combo_logsum_compare_2050.groupby(['SD_NAME', 'diff_tag'])['PARCEL_ID'].count().reset_index()

# project area
SD_stats_2050.loc[SD_stats_2050.SD_NAME.isin(['San Jose CBD', 'West San Jose', 'East San Jose',
                                                   'South San Jose', 'SE Snta Clara Cnty'])]

,SD_NAME,diff_tag,PARCEL_ID
8,East San Jose,P < NP,90734
9,East San Jose,P > NP,4
34,SE Snta Clara Cnty,P < NP,1697
35,SE Snta Clara Cnty,P = NP,6
36,SE Snta Clara Cnty,P > NP,32334
43,San Jose CBD,P < NP,63961
44,San Jose CBD,P > NP,1
50,South San Jose,P < NP,55060
51,South San Jose,P > NP,4563
59,West San Jose,P < NP,91425


# 5. export for mapping

## 5.1 use replicated calculations

In [40]:
M_GIS_dir = 'M:\\Data\\GIS layers\\UrbanSim smelt\\p10_EIR'
parcel_gdf_raw = gpd.read_file(os.path.join(M_GIS_dir, 'p10_parcels.shp'))
parcel_gdf = parcel_gdf_raw[['PARCEL_ID', 'ZONE_ID', 'geometry']]

In [41]:
parcel_combo_logsum_compare_2035_gdf = parcel_gdf.merge(parcel_combo_logsum_compare_2035,
                                                        on='PARCEL_ID', how='outer')
print(parcel_combo_logsum_compare_2035_gdf.shape[0])
parcel_combo_logsum_compare_2035_gdf.to_file(
    r'M:\Data\Urban\BAUS\PBA50\explore_developer_model\runs_new_parcel_subzone_input\trace\parcel_combo_logsum_compare_2035.geojson',
    driver='GeoJSON')

1956208


In [42]:
parcel_combo_logsum_compare_2050_gdf = parcel_gdf.merge(parcel_combo_logsum_compare_2050,
                                                        on='PARCEL_ID', how='outer')
print(parcel_combo_logsum_compare_2050_gdf.shape[0])
parcel_combo_logsum_compare_2050_gdf.to_file(
    r'M:\Data\Urban\BAUS\PBA50\explore_developer_model\runs_new_parcel_subzone_input\trace\parcel_combo_logsum_compare_2050.geojson',
    driver='GeoJSON')

1956208


## 5.2 use BAUS output

In [76]:
baus_output_2015_P_tag = baus_output_2015_P.copy()
baus_output_2015_NP_tag = baus_output_2015_NP.copy()

baus_output_2030_P_tag = baus_output_2030_P.copy()
baus_output_2030_NP_tag = baus_output_2030_NP.copy()

In [77]:
baus_output_2015_P_tag.columns = ['parcel_id_baus'] + [x+'_P' for x in list(baus_output_2015_P_tag)[1:]]
baus_output_2015_NP_tag.columns = ['parcel_id_baus'] + [x+'_NP' for x in list(baus_output_2015_NP_tag)[1:]]

baus_output_2030_P_tag.columns = ['parcel_id_baus'] + [x+'_P' for x in list(baus_output_2030_P_tag)[1:]]
baus_output_2030_NP_tag.columns = ['parcel_id_baus'] + [x+'_NP' for x in list(baus_output_2030_NP_tag)[1:]]

In [78]:
baus_output_2015_compare = baus_output_2015_P_tag.merge(baus_output_2015_NP_tag, on='parcel_id_baus', how='outer')
baus_output_2015_compare['diff'] = \
    baus_output_2015_compare['combo_logsum_baus_P'] - baus_output_2015_compare['combo_logsum_baus_NP']

baus_output_2015_compare['diff_tag'] = 'P > NP'
baus_output_2015_compare.loc[baus_output_2015_compare['diff'] < 0, 'diff_tag'] = 'P < NP'
baus_output_2015_compare.loc[baus_output_2015_compare['diff'] == 0, 'diff_tag'] = 'P = NP'

baus_output_2015_compare = baus_output_2015_compare.merge(parcel_subzone_lookup,
                                                          left_on='parcel_id_baus', right_on='PARCEL_ID',how='outer')

In [79]:
baus_output_2030_compare = baus_output_2030_P_tag.merge(baus_output_2030_NP_tag, on='parcel_id_baus', how='outer')
baus_output_2030_compare['diff'] = \
    baus_output_2030_compare['combo_logsum_baus_P'] - baus_output_2030_compare['combo_logsum_baus_NP']

baus_output_2030_compare['diff_tag'] = 'P > NP'
baus_output_2030_compare.loc[baus_output_2030_compare['diff'] < 0, 'diff_tag'] = 'P < NP'
baus_output_2030_compare.loc[baus_output_2030_compare['diff'] == 0, 'diff_tag'] = 'P = NP'

baus_output_2030_compare = baus_output_2030_compare.merge(parcel_subzone_lookup,
                                                          left_on='parcel_id_baus', right_on='PARCEL_ID',how='outer')

In [80]:
baus_output_2015_compare_gdf = parcel_gdf[['PARCEL_ID', 'geometry']].merge(baus_output_2015_compare,
                                                on='PARCEL_ID', how='outer')
print(baus_output_2015_compare_gdf.shape[0])

baus_output_2030_compare_gdf = parcel_gdf[['PARCEL_ID', 'geometry']].merge(baus_output_2030_compare,
                                                on='PARCEL_ID', how='outer')
print(baus_output_2030_compare_gdf.shape[0])

1956212
1956212


In [82]:
type(baus_output_2015_compare_gdf)

geopandas.geodataframe.GeoDataFrame

In [83]:
baus_output_2015_compare_gdf.to_file(
    r'M:\Data\Urban\BAUS\PBA50\explore_developer_model\runs_new_parcel_subzone_input\trace\logsum_baus_output_2015_compare_gdf.geojson',
    driver='GeoJSON')

baus_output_2030_compare_gdf.to_file(
    r'M:\Data\Urban\BAUS\PBA50\explore_developer_model\runs_new_parcel_subzone_input\trace\logsum_baus_output_2030_compare_gdf.geojson',
    driver='GeoJSON')